In [ ]:
#==================================================================
#Program: SLADS_TensorFlow_Simulation
#Author(s): David Helminiak
#Date Created: 13 February 2019
#Date Last Modified: May 2019
#Changelog: 0.1  - Combined Structure             - February 2019
#           0.2  - Combined Train/Test            - February 2019
#           0.3  - Gaussian CPU Multi-Threading   - February 2019
#           0.4  - Restructuring of Dirs          - March 2019
#           0.5  - Clearer code progress viz      - March 2019
#           0.6  - Plotting Statistics            - March 2019
#           0.61 - Initial/Var Mask Generation    - May 2019           
#           0.7  - Line Scanning                  - May 2019
#           0.71 - Partial Line Scanning          - June 2019
#           0.72 - Dynamic Partial Line Scanning  - June 2019
#           0.73 - Code Simplification            - July 2019
#           0.74 - Prediction Improvements        - July 2019
#           0.75 - Sparse Grid                    - July 2019
#           0.8  - .RAW Integration
#           0.9  - Continuous Value Prediction
#==================================================================
#==================================================================

#==================================================================
#ADDITIONAL NOTES:
#==================================================================
#Add Breakpoint anywhere in the program: 
#Tracer()()
#
#For jupyter-notebook, resize cells to fit to length of window
#from IPython.core.display import display, HTML
#display(HTML("<style>.container {width:80% !important;}</style>"))
#
#NOTE - MeasuredIdxs does not contain the initial mask idxs
#
#==================================================================
#==================================================================

#==================================================================
#LIBRARY IMPORTS
#==================================================================
from __future__ import absolute_import, division, print_function
#import tensorflow as tf
#tf.enable_eager_execution() #Evaluate all operations without building graphs
import cv2
import pandas as pd
import numpy as np
import numpy.matlib as matlib
import matplotlib.pyplot as plt
import multiprocessing
import os
import PIL
import PIL.ImageOps  
import math
import natsort
import glob
import re
import random
import sys
import scipy
import shutil
import time
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
from itertools import zip_longest
from IPython import display
from IPython.core.debugger import Tracer
from joblib import Parallel, delayed
from matplotlib.pyplot import figure
from PIL import Image
from scipy import misc
from scipy.io import loadmat
from scipy.io import savemat
from sklearn import linear_model
from sklearn.utils import shuffle
from skimage.util import view_as_windows as viewW
from sklearn.neighbors import NearestNeighbors
from skimage.measure import compare_ssim
from skimage import filters
from sobol import *
from tqdm.auto import tqdm

#from tensorflow import keras
#==================================================================

In [2]:
#==================================================================
#CLASS AND FUNCTION DEFINITIONS
#==================================================================


#Return all of the possible lines for an np array of angle values
def allLinesForAngle(manualAngles):
    linesToScan = []
    for manualAngle in manualAngles:
        #Compensate for mask being upside down and reversed
        theta = manualAngle+90

        #Make a list of all the possible lines in the image for a particular angle
        if theta < 0:
            theta = theta+360
        if theta > 180:
            theta = theta - 180*int(theta/180)
        if theta > 90:
            theta = 90-(theta-90)
            flipX = True
        else:
            flipX = False
        yfact = math.sin(math.radians(theta))
        xfact = math.cos(math.radians(theta))

        #Make the initial line
        currX = 0
        currY = 0
        pts = []
        while currX <= SizeImage[0] and currY <= SizeImage[1]:
            if currX < SizeImage[0] and currY < SizeImage[1] and currX >= 0 and currY >= 0:
                pts.append([int(currX), int(currY)])
            currX = currX + xfact
            currY = currY + yfact
        pts = np.asarray(pd.DataFrame(pts).drop_duplicates())

        if flipX:
            pts[:,0] = np.flip(pts[:,0])

        if theta <= 45: #theta is less, or equal to 45 deg
            #Each column should have only a single value such that shifting does not create duplicate points to scan between >45 and 135
            simppts = []
            for i in range(0,SizeImage[0]):
                columnIdxs = []
                for j in range(0,len(pts)):
                    if pts[j][0] == i:
                        columnIdxs.append(pts[j].tolist())
                if len(columnIdxs) > 1:
                    simppts.append(columnIdxs[np.argmin(np.asarray(columnIdxs)[:,1])])
                else:
                    simppts.append(columnIdxs[0])

            #Shift the line along y until there are no points in range
            simppts = np.asarray(simppts)
            currY = -SizeImage[1]
            while currY <= SizeImage[1]:
                temppts = simppts.copy()
                temppts[:,1] = temppts[:,1]+currY
                line = []
                for i in range(0,len(temppts)):
                    xpt, ypt = temppts[i][0], temppts[i][1]
                    if xpt < SizeImage[0] and ypt < SizeImage[1] and xpt >= 0 and ypt >= 0:
                        line.append([xpt, ypt])
                if len(line) > 0:
                    linesToScan.append(line)
                currY = currY + 1

        else: #theta is greater than 45 deg
            #Each row should have only a single value
            simppts = []
            for i in range(0,SizeImage[0]):
                rowIdxs = []
                for j in range(0,len(pts)):
                    if pts[j][1] == i:
                        rowIdxs.append(pts[j].tolist())
                if len(rowIdxs) > 1:
                    simppts.append(rowIdxs[np.argmin(np.asarray(rowIdxs)[:,0])])
                else:
                    simppts.append(rowIdxs[0])

            #Shift the line along x until there are no points in range
            simppts = np.asarray(simppts)
            currX = -SizeImage[0]
            while currX <= SizeImage[0]:
                temppts = simppts.copy()
                temppts[:,0] = temppts[:,0]+currX
                line = []
                for i in range(0,len(temppts)):
                    xpt, ypt = temppts[i][0], temppts[i][1]
                    if xpt < SizeImage[0] and ypt < SizeImage[1] and xpt >= 0 and ypt >= 0:
                        line.append([xpt, ypt])
                if len(line) > 0:
                    linesToScan.append(line)
                currX = currX + 1
    return linesToScan


def plotErrorData(savePlotLocation, savePlotSuffix, trainingResultsAverageObject, xAxisValues, plotTitle, plotXLabel): #Plot and save the error data obtained during training

    #Format the base plot
    font = {'size' : 18}
    plt.rc('font', **font)

    #Add plot
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()

    #Add error plots as training data is recorded
    if (len(trainingResultsAverageObject.mseAverageErrors) > 0):
        plt.plot(xAxisValues, trainingResultsAverageObject.mseAverageErrors, color='black', linestyle='--') 
        plt.scatter(xAxisValues, trainingResultsAverageObject.mseAverageErrors, color='black') 

    #Label plot
    plt.title(plotTitle)
    plt.xlabel(plotXLabel)
    plt.ylabel('Average MSE')

    #Export image
    plt.savefig(savePlotLocation + 'MSE' + savePlotSuffix + '.png')
    plt.close()
    
    #Add plot
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()

    #Add error plots as training data is recorded
    if (len(trainingResultsAverageObject.ssimAverageErrors) > 0):
        plt.plot(xAxisValues, trainingResultsAverageObject.ssimAverageErrors, color='black', linestyle='--') 
        plt.scatter(xAxisValues, trainingResultsAverageObject.ssimAverageErrors, color='black') 

    #Label plot
    plt.title(plotTitle)
    plt.xlabel(plotXLabel)
    plt.ylabel('Average SSIM')

    #Export image
    plt.savefig(savePlotLocation + 'SSIM' + savePlotSuffix + '.png')
    plt.close()
    
    #Add plotIm
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()

    #Add error plots as training data is recorded
    if (len(trainingResultsAverageObject.distortionAverageErrors) > 0):
        plt.plot(xAxisValues, trainingResultsAverageObject.distortionAverageErrors, color='black', linestyle='--') 
        plt.scatter(xAxisValues, trainingResultsAverageObject.distortionAverageErrors, color='black') 

    #Label plot
    plt.title(plotTitle)
    plt.xlabel(plotXLabel)
    plt.ylabel('Average % Total Distortion')

    #Export image
    plt.savefig(savePlotLocation + 'TotalDistortion' + savePlotSuffix + '.png')
    plt.close()

def ttPlotAverageErrors(savePlotLocation, StopPercentageTestingSLADSArr, trainTestAverageErrors):

    ttMSE = []
    ttSSIM = []
    ttTD = []

    for i in range(0, len(trainTestAverageErrors)):
        ttMSE.append(trainTestAverageErrors[i].mseAverageErrors)
        ttSSIM.append(trainTestAverageErrors[i].ssimAverageErrors)
        ttTD.append(trainTestAverageErrors[i].distortionAverageErrors)

    xAxisValues = StopPercentageTestingSLADSArr.tolist()
    #TD
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()
    if (len(ttTD) > 0):
        plt.plot(xAxisValues, ttTD, color='black') 
        plt.scatter(xAxisValues, ttTD, color='black') 
    plt.xlabel('% Sampled')
    plt.ylabel('Average Total Distortion')
    plt.savefig(savePlotLocation + 'ttPlotAverageTD' + '.png')
    plt.close()

    #MSE
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()
    if (len(ttMSE) > 0):
        plt.plot(xAxisValues, ttMSE, color='black') 
        plt.scatter(xAxisValues, ttMSE, color='black') 
    plt.xlabel('% Sampled')
    plt.ylabel('Average MSE')
    plt.savefig(savePlotLocation + 'ttPlotAverageMSE' + '.png')
    plt.close()
    
    #SSIM
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()
    if (len(ttSSIM) > 0):
        plt.plot(xAxisValues, ttSSIM, color='black') 
        plt.scatter(xAxisValues, ttSSIM, color='black') 
    plt.xlabel('% Sampled')
    plt.ylabel('Average SSIM')
    plt.savefig(savePlotLocation + 'ttPlotAverageSSIM' + '.png')
    plt.close()
    
class SLADSResults: #Object to hold and output run results 
    def __init__(self):
        
        #Values recorded for each line scanned
        self.tdValues = []
        self.mseValues = []
        self.ssimValues = []
        self.tdValues_forImage = []
        self.mseValues_forImage = []
        self.ssimValues_forImage = []
        self.percentageValues = []
        self.Masks = []
        self.ReconImages = []
        self.ERDValuesNPs = []
        self.animationSavePaths = []
        self.timesToDetermine = []
        self.img = None
        self.forimg = None
        
    def imgInit(self,img):
        self.img = img
        self.forimg = cv2.bitwise_and(img,img,mask = ((img >= filters.threshold_otsu(img))+0).astype('uint8'))

    def saveData(self, SavePath, fileName, percMeasured, NumSamples, pixelCount, SizeImage, Mask, ReconImage, ERDValuesNP, ImageType, animationFlag, simulationFlag, testingFlag, completedRunFlag, noPointsFlag, trainingPlotFlag):
        ImageSavePath = SavePath + 'ImageResults' + os.path.sep
        if not noPointsFlag:
            nextPercFlag = False
            percMeasured = ((NumSamples/pixelCount)*100)

            #If this is the first set of values, or if the percentage has increased at least a percent
            #if (len(self.percentageValues) == 0) or (percMeasured >= self.percentageValues[len(self.percentageValues)-1]):
            #    nextPercFlag = True

            self.percentageValues.append(percMeasured)
            self.Masks.append(Mask.copy())
            self.ReconImages.append(ReconImage.copy())
            self.ERDValuesNPs.append(ERDValuesNP.copy())
            self.animationSavePaths.append(ImageSavePath + fileName + 'lines_' + str(len(self.percentageValues)) + '.png')

            if (testingFlag and simulationFlag) or trainingPlotFlag:

                #Find statistics of interest
                Difference = np.sum(computeDifference(self.img,ReconImage,ImageType))
                TD = Difference/(SizeImage[0]*SizeImage[1])
                MSE = (np.sum((ReconImage.astype("float") - self.img.astype("float")) ** 2))/(float(ReconImage.shape[0] * ReconImage.shape[1]))
                SSIM = compare_ssim(ReconImage.astype("float"), self.img.astype("float"))

                Difference_forImage = np.sum(computeDifference(self.forimg,ReconImage,ImageType))
                TD_forImage = Difference_forImage/(SizeImage[0]*SizeImage[1])
                MSE_forImage = (np.sum((ReconImage.astype("float") - self.forimg.astype("float")) ** 2))/(float(ReconImage.shape[0] * ReconImage.shape[1]))
                SSIM_forImage = compare_ssim(ReconImage.astype("float"), self.forimg.astype("float"))

                #Save them internally
                self.tdValues.append(TD)
                self.mseValues.append(MSE)
                self.ssimValues.append(SSIM)
                self.tdValues_forImage.append(TD_forImage)
                self.mseValues_forImage.append(MSE_forImage)
                self.ssimValues_forImage.append(SSIM_forImage)

                #If the current percentage measured is at least one percentage larger than the last, or if 
                #if percMeasured >= self.percentageValues[len(self.percentageValues)-2]

        if completedRunFlag:
            #Normalize the ERDValuesNP arrays across entire series of images
            ERDValuesNPMax = 0
            for ERDValuesNP in self.ERDValuesNPs:
                if np.max(ERDValuesNP) > ERDValuesNPMax:
                    ERDValuesNPMax = np.max(ERDValuesNP)
            for ERDValuesNP in self.ERDValuesNPs:
                ERDValuesNP *= (255.0/ERDValuesNPMax)
             
        #If a simulation test has finished
        if simulationFlag and testingFlag and completedRunFlag and not trainingPlotFlag:
            for i in range(0,len(self.Masks)):
                TD = self.tdValues[i]
                SSIM = self.ssimValues[i]
                MSE = self.mseValues[i]
                
                TD_forImage = self.tdValues_forImage[i]
                SSIM_forImage = self.ssimValues_forImage[i]
                MSE_forImage = self.mseValues_forImage[i]
                
                percMeasured = self.percentageValues[i]
                Mask = self.Masks[i]
                ERDValuesNP = self.ERDValuesNPs[i]
                ReconImage = self.ReconImages[i]
                animationSavePath = self.animationSavePaths[i]
                
                font = {'size' : 18}
                plt.rc('font', **font)
                f = plt.figure(figsize=(15,15))

                f.subplots_adjust(top = 0.85)
                f.subplots_adjust(wspace=0.15, hspace=0.2)
                plt.suptitle("Percent Sampled: %.2f, Measurement Iteration: %.0f\nSSIM: %.2f, TD: %.2f" % (percMeasured, i+1, SSIM, TD), fontsize=20, fontweight='bold', y = 0.95)
                sub = f.add_subplot(2,2,1)
                sub.imshow(self.img, cmap='hot', vmin=0, vmax=255)
                sub.set_title('Ground-Truth')

                sub = f.add_subplot(2,2,2)
                sub.imshow(ReconImage, cmap='hot', vmin=0, vmax=255)
                sub.set_title('Reconstructed Image')

                sub = f.add_subplot(2,2,3)
                sub.imshow(Mask, cmap='gray')
                sub.set_title('Sampled Mask')

                sub = f.add_subplot(2,2,4)
                im = sub.imshow(ERDValuesNP, cmap='viridis', vmin=0, vmax=255)
                sub.set_title('ERD Values')
                cbar = f.colorbar(im, ax=sub, orientation='vertical', pad=0.01)



#Plotting arrangement for with foreground isolation
#                 font = {'size' : 18}
#                 plt.rc('font', **font)
#                 f = plt.figure(figsize=(20,12))
#                 f.subplots_adjust(top = 0.82)
#                 f.subplots_adjust(wspace=0.15, hspace=0.2)
#                 #plt.suptitle("Percent Sampled: %.2f\nGround-Truth Image - MSE: %.2f, SSIM: %.2f, TD: %.2f\nForeground Image - MSE: %.2f, SSIM: %.2f, TD: %.2f" % (percMeasured, MSE, SSIM, TD, MSE_forImage, SSIM_forImage, TD_forImage), fontsize=20, fontweight='bold', y = 0.95)
                
#                 sub = f.add_subplot(2,4,(2,3))
#                 sub.imshow(self.img, cmap='hot', vmin=0, vmax=255)
#                 sub.set_title('Ground-Truth')

#                 #sub = f.add_subplot(2,4,(3,4))
#                 #sub.imshow(self.forimg, cmap='hot', vmin=0, vmax=255)
#                 #sub.set_title('Foreground')
                
#                 sub = f.add_subplot(2,3,4)
#                 sub.imshow(Mask, cmap='gray')
#                 sub.set_title('Sampled Mask')

#                 sub = f.add_subplot(2,3,5)
#                 sub.imshow(ReconImage, cmap='hot', vmin=0, vmax=255)
#                 sub.set_title('Reconstructed Image')

#                 sub = f.add_subplot(2,3,6)
#                 im = sub.imshow(ERDValuesNP, cmap='viridis', vmin=0, vmax=255)
#                 sub.set_title('ERD Values')
#                 cbar = f.colorbar(im, ax=sub, orientation='vertical', pad=0.1)

                plt.savefig(animationSavePath, bbox_inches='tight')
                plt.close()
            
        #If not testing or simulation run, but the program has completed
        elif not testingFlag and not simulationFlag and completedRunFlag:
            for i in range(0,len(self.Masks)):
                percMeasured = self.percentageValues[i]
                Mask = self.Masks[i]
                ERDValuesNP = self.ERDValuesNPs[i]
                ReconImage = self.ReconImages[i]
                animationSavePath = self.animationSavePaths[i]
                font = {'size' : 18}
                plt.rc('font', **font)
                f = plt.figure(figsize=(30,8))
                plt.suptitle("Percent Sampled: %.2f" % percMeasured, fontsize=20, fontweight='bold', y = 0.9)
                ax1 = f.add_subplot(131)
                ax1.imshow(Mask, cmap='hot')
                ax1.set_title('Sampled Mask')
                ax2 = f.add_subplot(132)
                ax2.imshow(ERDValuesNP)
                ax2.set_title('ERD Values')
                ax3 = f.add_subplot(133)
                ax3.imshow(ReconImage, cmap='hot', vmin=0, vmax=255)
                ax3.set_title('Reconstructed Image')
                plt.savefig(animationSavePath)
                plt.close()
                
        #When an animation image set has completed generation
        if animationFlag and testingFlag and completedRunFlag:
            #Move animation image set into its own directory and form video from them
            animationPath = ImageSavePath+'Animation'+os.path.sep+fileName
            os.makedirs(animationPath)
            dataFileNames = natsort.natsorted(glob.glob(ImageSavePath + "/"+fileName+"*"+ImageExtension))
            height, width, layers = cv2.imread(dataFileNames[0]).shape
            animation = cv2.VideoWriter(animationPath+'.avi',cv2.VideoWriter_fourcc(*'MJPG'), 2, (width, height))
            for specFileName in dataFileNames: #For each of the files
                animation.write(cv2.imread(specFileName))
                shutil.move(specFileName, animationPath+os.path.sep+os.path.basename(specFileName))
            animation.release()
            animation = None
        
#Storage location for training information definition
class TrainingInfo:
    def initialize(self,ReconMethod,FeatReconMethod, p, NumNbrs, FilterType, FilterC, FeatDistCutoff, MaxWindowForTraining,*args):
        self.ReconMethod = ReconMethod
        self.FeatReconMethod = FeatReconMethod
        self.p = p
        self.NumNbrs = NumNbrs
        self.FilterType = FilterType
        self.FilterC = FilterC
        self.FeatDistCutoff = FeatDistCutoff
        self.MaxWindowForTraining=MaxWindowForTraining
        if args:
            self.PAP_Iter=args[0]
            self.PAP_Beta=args[1]
            self.PAP_InitType=args[2]
            self.PAP_ScaleMax=args[3]
        
class TestingInfo:
    def initialize(self):
        self.mseTestingError = [] #Storage location for MSE testing error data
        self.ssimTestingError = [] #Storage location for SSIM testing error data
    def saveTestingErrorData(self, mseValue, ssimValue): #Store training error information
        self.mseTestingError.append(mseValue)
        self.ssimTestingError.append(ssimValue)
    def plotTestingErrorData(self, savePlotLocation): #Plot and save the error data obtained during training
        plt.savefig(savePlotLocation + 'testingStatistics.png')    
        plt.close()
        
#Storage location for the initial mask definition
class InitialMask:
    def initialize(self,RowSz,ColSz,MaskType,MaskNumber,Percentage,originalLinesToScan,linesToScan,initialMaskPts,useERDmean,partialLine):
        self.RowSz = RowSz
        self.ColSz = ColSz
        self.MaskType = MaskType
        self.MaskNumber = MaskNumber
        self.Percentage = Percentage
        self.originalLinesToScan = originalLinesToScan
        self.linesToScan = linesToScan
        self.initialMaskPts = initialMaskPts
        self.useERDmean = useERDmean
        self.partialLine = partialLine
    def delLine(self, index):
        self.linesToScan = np.delete(self.linesToScan, index, 0)
    def delPoints(self, pts):
        for i in range(0,len(self.linesToScan)):
            indexes = []
            for pt in pts:
                indexes.append([i for i, j in enumerate(self.linesToScan[i]) if j == pt])
            indexes = [x for x in np.asarray(indexes).flatten().tolist() if x != []]
            if len(indexes) > 0:
                self.linesToScan[i] = np.delete(self.linesToScan[i], indexes,0).tolist()

#Storage location for the training stopping parameters
class StopCondParams:
    def initialize(self,Beta,Threshold,JforGradient,MinPercentage,MaxPercentage):
        self.Beta = Beta
        self.Threshold = Threshold
        self.JforGradient = JforGradient
        self.MinPercentage = MinPercentage
        self.MaxPercentage = MaxPercentage

#Storage location for ERD parameters
class UpdateERDParams:
    def initialize(self,Do,MinRadius,MaxRadius,IncreaseRadiusBy):
        self.Do = Do
        self.MinRadius = MinRadius
        self.MaxRadius = MaxRadius
        self.IncreaseRadiusBy = IncreaseRadiusBy

#Storage location for batch sampling parameters
class BatchSamplingParams:
    def initialize(self,NumSamplesPerIter):
        self.NumSamplesPerIter = NumSamplesPerIter

#Storage object for image data
class imageData:
    def __init__(self, data, name):
        self.name =name
        self.data = data
        
#Convert an image into a series of columns
def im2col(Matrix,WidowSize):
    M,N = Matrix.shape
    col_extent = N - WidowSize[1] + 1
    row_extent = M - WidowSize[0] + 1
    start_idx = np.arange(WidowSize[0])[:,None]*N + np.arange(WidowSize[1])
    offset_idx = np.arange(row_extent)[:,None]*N + np.arange(col_extent)
    out = np.take (Matrix,start_idx.ravel()[:,None] + offset_idx.ravel())
    return(out)
    # http://stackoverflow.com/questions/30109068/implement-matlabs-im2col-sliding-in-python

def generateGaussianKernel(sigma,WindowSize):
    FilterMat = np.ones((WindowSize[0],WindowSize[1]))
    for i in range(0,WindowSize[0]):
        for j in range(0,WindowSize[1]):
            FilterMat[i][j]=np.exp( -(1/(2*sigma**2)) * np.absolute( ( (i-np.floor(WindowSize[0]/2))**2 +  (j-np.floor(WindowSize[1]/2))**2 ) )  )
    FilterMat = FilterMat/np.amax(FilterMat)
    FilterMat = np.transpose(FilterMat)
    Filter=np.ravel(FilterMat)
    return Filter

def computeFullERD(MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType):
    
    NeighborValues,NeighborWeights,NeighborDistances = FindNeighbors(TrainingInfoObject,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Resolution)

    ReconValues,ReconImage = ComputeRecons(TrainingInfoObject,NeighborValues,NeighborWeights,SizeImage,UnMeasuredIdxs,MeasuredIdxs,MeasuredValues)
    
    # Compute features
    PolyFeatures=computeFeatures(UnMeasuredIdxs,SizeImage,NeighborValues,NeighborWeights,NeighborDistances,TrainingInfoObject,ReconValues,ReconImage,Resolution,ImageType)
    
    # Compute ERD
    ERDValues = PolyFeatures.dot(Theta)
    
    return(ERDValues,ReconValues,ReconImage)

def updateERD(Mask,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,NewIdxs,NumSamples,UpdateERDParamsObject,ReconValues,ReconImage,ERDValues,MaxIdxsVect,BatchSamplingParamsObject):

    ERDValues=np.delete(ERDValues,(MaxIdxsVect))
    ReconValues=np.delete(ReconValues,(MaxIdxsVect))
    SuggestedRadius = int(np.sqrt((1/np.pi)*(SizeImage[0]*SizeImage[1]*TrainingInfoObject.NumNbrs/NumSamples)))
    UpdateRadiusTemp=np.max([SuggestedRadius,UpdateERDParamsObject.MinRadius]);
    UpdateRadius=int(np.min([UpdateERDParamsObject.MaxRadius,UpdateRadiusTemp]));

    updateRadiusMat = np.zeros((SizeImage[0],SizeImage[1]))
    Done=0
    while(Done==0):
        for b in range(0,len(NewIdxs)):
            updateRadiusMat[max(NewIdxs[b][0]-UpdateRadius,0):min(NewIdxs[b][0]+UpdateRadius,SizeImage[0])][:,max(NewIdxs[b][1]-UpdateRadius,0):min(NewIdxs[b][1]+UpdateRadius,SizeImage[1])]=1
    
        updateIdxs = np.where(updateRadiusMat[Mask==0]==1)
        
        SmallUnMeasuredIdxs = np.transpose(np.where(np.logical_and(Mask==0,updateRadiusMat==1)))
        if SmallUnMeasuredIdxs.size==0:
            UpdateRadius=int(UpdateRadius*UpdateERDParamsObject.IncreaseRadiusBy)
        else:
            Done=1
    
    # Find neighbors of unmeasured locations
    SmallNeighborValues,SmallNeighborWeights,SmallNeighborDistances = FindNeighbors(TrainingInfoObject,MeasuredIdxs,SmallUnMeasuredIdxs,MeasuredValues,Resolution)
    
    # Perform reconstruction
    SmallReconValues=computeWeightedMRecons(SmallNeighborValues,SmallNeighborWeights,TrainingInfoObject)
    
    ReconImage[(np.logical_and(Mask==0,updateRadiusMat==1))]=SmallReconValues
    ReconImage[MeasuredIdxs[:,0],MeasuredIdxs[:,1]]=MeasuredValues

    # Compute features
    SmallPolyFeatures=computeFeatures(SmallUnMeasuredIdxs,SizeImage,SmallNeighborValues,SmallNeighborWeights,SmallNeighborDistances,TrainingInfoObject,SmallReconValues,ReconImage,Resolution,ImageType)

    # Compute ERD
    SmallERDValues = SmallPolyFeatures.dot(Theta)
    ReconValues[updateIdxs[0]] = SmallReconValues
    ERDValues[updateIdxs] = SmallERDValues

    return(ERDValues,ReconValues)

def MakeERDArray(ERDValues, SizeImage, UnMeasuredIdxs, ReconImage):
    #Rearrange ERD values into array; those that have already been measured have 0 ERD
    ERDValuesNP = np.zeros([SizeImage[0],SizeImage[1]])
    
    #Copy over ERD values for unmeasured points
    for i in range(0, len(UnMeasuredIdxs)): ERDValuesNP[UnMeasuredIdxs[i][0], UnMeasuredIdxs[i][1]] = ERDValues[i]

    #Remove values that are less than those already scanned (0 ERD)
    ERDValuesNP[np.where((ERDValuesNP < 0))] = 0
    
    #The following masks the ERD Values array by the reconstruction image
    #Dr. Ye has requested that this masking procedure, which limits scanning by the determined structure
    #be deactivated
    #ERDValuesNP[np.where((ReconImage < np.std(ReconImage)))] = 0
    
    return ERDValuesNP

def FindNeighbors(TrainingInfoObject,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Resolution):

    # Find neighbors of unmeasured locations
    Neigh = NearestNeighbors(n_neighbors=TrainingInfoObject.NumNbrs)
    Neigh.fit(MeasuredIdxs)
    NeighborDistances, NeighborIndices = Neigh.kneighbors(UnMeasuredIdxs)
    NeighborDistances=NeighborDistances*Resolution
    NeighborValues=MeasuredValues[NeighborIndices]
    NeighborWeights=computeNeighborWeights(NeighborDistances,TrainingInfoObject)
    
    return(NeighborValues,NeighborWeights,NeighborDistances) 

def ComputeRecons(TrainingInfoObject,NeighborValues,NeighborWeights,SizeImage,UnMeasuredIdxs,MeasuredIdxs,MeasuredValues):
    # Perform reconstruction
    ReconValues=computeWeightedMRecons(NeighborValues,NeighborWeights,TrainingInfoObject)
    ReconImage = np.zeros((SizeImage[0],SizeImage[1]))
    ReconImage[UnMeasuredIdxs[:,0],UnMeasuredIdxs[:,1]]=ReconValues
    ReconImage[MeasuredIdxs[:,0],MeasuredIdxs[:,1]]=MeasuredValues
    return(ReconValues,ReconImage)

def computeNeighborWeights(NeighborDistances,TrainingInfoObject):
    
    UnNormNeighborWeights=1/np.power(NeighborDistances,TrainingInfoObject.p)
    SumOverRow = (np.sum(UnNormNeighborWeights,axis=1))
    NeighborWeights=UnNormNeighborWeights/SumOverRow[:, np.newaxis]
    return NeighborWeights

def computeWeightedMRecons(NeighborValues,NeighborWeights,TrainingInfoObject):
    
    # Weighted Mode Computation
    if TrainingInfoObject.FeatReconMethod=='DWM':
        ClassLabels = np.unique(NeighborValues)
        ClassWeightSums = np.zeros((np.shape(NeighborWeights)[0],np.shape(ClassLabels)[0]))
        for i in range(0,np.shape(ClassLabels)[0]):
            TempFeats=np.zeros((np.shape(NeighborWeights)[0],np.shape(NeighborWeights)[1]))
            np.copyto(TempFeats,NeighborWeights)
            TempFeats[NeighborValues!=ClassLabels[i]]=0
            ClassWeightSums[:,i]=np.sum(TempFeats,axis=1)
        IdxOfMaxClass = np.argmax(ClassWeightSums,axis=1)
        ReconValues = ClassLabels[IdxOfMaxClass]

    # Weighted Mean Computation
    elif TrainingInfoObject.FeatReconMethod=='CWM':
        ReconValues=np.sum(NeighborValues*NeighborWeights,axis=1)

    return ReconValues

def computeDifference(array1,array2,type):
    if type == 'D':
        difference=array1!=array2
        difference = difference.astype(float)
    if type == 'C':
        difference=abs(array1-array2)
    return difference

def computeFeatures(UnMeasuredIdxs,SizeImage,NeighborValues,NeighborWeights,NeighborDistances,TrainingInfoObject,ReconValues,ReconImage,Resolution,ImageType):
    #12345
    #Feature=np.zeros((np.shape(UnMeasuredIdxs)[0],2))
    
    #Feature[:,0],Feature[:,1]=computeStDivFeatures(NeighborValues,NeighborWeights,TrainingInfoObject,ReconValues,ImageType)
    
    #GradientImageX,GradientImageY=computeGradientFeatures(ReconImage,ImageType)
    #Feature[:,0] = GradientImageY[UnMeasuredIdxs[:,0],UnMeasuredIdxs[:,1]]
    #Feature[:,1] = GradientImageX[UnMeasuredIdxs[:,0],UnMeasuredIdxs[:,1]]
    
    #CutoffDist = np.ceil(np.sqrt((TrainingInfoObject.FeatDistCutoff/100)*(SizeImage[0]*SizeImage[1]/np.pi)))
    #Feature[:,4] = NeighborDistances[:,0]
    
    #NeighborsInCircle=np.sum(NeighborDistances<=CutoffDist,axis=1)
    #Feature[:,4] = (1+(np.pi*(np.power(CutoffDist,2))))/(1+NeighborsInCircle)
    
    
    #Feature[:,4],Feature[:,5]=computeDensityDistanceFeatures(NeighborDistances,NeighborWeights,SizeImage,TrainingInfoObject,ReconValues,ImageType)
    
    #Feature[:,2] = NeighborDistances[:,0]
    #
    #CutoffDist = np.ceil(np.sqrt((TrainingInfoObject.FeatDistCutoff/100)*(SizeImage[0]*SizeImage[1]/np.pi)))
    #NeighborsInCircle=np.sum(NeighborDistances<=CutoffDist,axis=1)
    #Feature[:,3] = (1+(np.pi*(np.power(CutoffDist,2))))/(1+NeighborsInCircle)
    
    
    
    
    Feature=np.zeros((np.shape(UnMeasuredIdxs)[0],6))

    # Compute std div features
    Feature[:,0],Feature[:,1]=computeStDivFeatures(NeighborValues,NeighborWeights,TrainingInfoObject,ReconValues,ImageType)
    
    # Compute distance/density features
    Feature[:,2],Feature[:,3]=computeDensityDistanceFeatures(NeighborDistances,NeighborWeights,SizeImage,TrainingInfoObject,ReconValues,ImageType)

    # Compute gradient features
    GradientImageX,GradientImageY=computeGradientFeatures(ReconImage,ImageType)
    Feature[:,4] = GradientImageY[UnMeasuredIdxs[:,0],UnMeasuredIdxs[:,1]]
    Feature[:,5] = GradientImageX[UnMeasuredIdxs[:,0],UnMeasuredIdxs[:,1]]
    
    PolyFeatures = computePolyFeatures(Feature)
    return PolyFeatures

def computeGradientFeatures(ReconImage,ImageType):
    GradientImageX,GradientImageY = np.gradient(ReconImage)
    if ImageType=='D':
        GradientImageX[GradientImageX!=0]=1
        GradientImageY[GradientImageY!=0]=1
    elif ImageType=='C':
        GradientImageX=abs(GradientImageX)
        GradientImageY=abs(GradientImageY)
    return(GradientImageX,GradientImageY)

def computeStDivFeatures(NeighborValues,NeighborWeights,TrainingInfoObject,ReconValues,ImageType):
    DiffVect = computeDifference(NeighborValues,np.transpose(np.matlib.repmat(ReconValues,np.shape(NeighborValues)[1],1)),ImageType)
    Feature_0 = np.sum(NeighborWeights*DiffVect,axis=1)
    Feature_1 = np.sqrt((1/TrainingInfoObject.NumNbrs)*np.sum(np.power(DiffVect,2),axis=1))
    return(Feature_0,Feature_1)

def computeDensityDistanceFeatures(NeighborDistances,NeighborWeights,SizeImage,TrainingInfoObject,ReconValues,ImageType):
    CutoffDist = np.ceil(np.sqrt((TrainingInfoObject.FeatDistCutoff/100)*(SizeImage[0]*SizeImage[1]/np.pi)))
    Feature_2 = NeighborDistances[:,0]
    NeighborsInCircle=np.sum(NeighborDistances<=CutoffDist,axis=1)
    Feature_3 = (1+(np.pi*(np.power(CutoffDist,2))))/(1+NeighborsInCircle)
    return(Feature_2,Feature_3)

def computePolyFeatures(Feature):
    PolyFeatures = np.hstack([np.ones((np.shape(Feature)[0],1)),Feature])
    for i in range(0,np.shape(Feature)[1]):
        for j in range(i,np.shape(Feature)[1]):
            Temp = Feature[:,i]*Feature[:,j]
            PolyFeatures = np.column_stack([PolyFeatures,Feature[:,i]*Feature[:,j]])
    return PolyFeatures

def performSLADStoFindC(num_threads, consoleRunning, codePath,trainingDataPath,ImageSet,ImageType,
                        ImageExtension,TrainingInfoObject,SizeImage,StopPercentage,Resolution,c_vec,
                        UpdateERDParamsObject,InitialMaskObject,MaskType,reconPercVector,Classify,consoleRows,
                        cPlot,savePlotLocation,lineScanning,BatchSamplingParamsObject):
    sys.path.append('code')
    
    # Initialize stopping condition variable
    Beta = computeBeta(SizeImage)
    StopCondParamsObject = StopCondParams()
    StopCondParamsObject.initialize(Beta,0,50,2,StopPercentage)
    
    SavePathSLADS = trainingDataPath + 'SLADSResults' 
    PlotResult = 'N'

    if not os.path.exists(SavePathSLADS):
        os.makedirs(SavePathSLADS)
        
    AreaUnderCurve = np.zeros(c_vec.shape[0])
    Idx_c = 0
    
    loadPathImage = trainingDataPath + 'Images' + os.path.sep
    loadPathInitialMask = resultsDataPath + 'InitialSamplingMasks'
    imageNames = glob.glob(loadPathImage + '*' + ImageExtension)
    NumImages = np.size(imageNames)
    
    for i sin tqdm(range(0, len(c_vec)), desc = 'c Values', leave = True): #For each of the proposed c values
        c = c_vec[i]
        LoadPath_c = trainingDataPath + 'c_' + str(c)
        TrainingInfoObject.FilterC=c
        Theta = np.load(LoadPath_c + os.path.sep + 'Theta' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy')
        for ImNum in tqdm(range(0, NumImages), desc = 'Images', leave = True): #For each of the images
            img = misc.imread(imageNames[ImNum])
            SavePathSLADS_c_ImNum = SavePathSLADS +  os.path.sep + 'Image_' + str(ImNum) + '_c_'+ str(c)
            if not os.path.exists(SavePathSLADS_c_ImNum):
                os.makedirs(SavePathSLADS_c_ImNum) 

            SavePath = SavePathSLADS + os.path.sep + 'Image_' + str(ImNum) + '_c_'+ str(c) + os.path.sep
            isRunningParallel = True
            runSLADS(InitialMaskObject,loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning),codePath,ImageSet,SizeImage,StopCondParamsObject,Theta,TrainingInfoObject,TestingInfoObject,Resolution,ImageType,UpdateERDParamsObject,BatchSamplingParamsObject,SavePath,'','',ImNum,ImageExtension,Classify,isRunningParallel,lineScanning,img,False,False,True, False)
            
            MeasuredValuesFull=np.load(SavePath + 'MeasuredValues.npy')
            MeasuredIdxsFull=np.load(SavePath + 'MeasuredIdxs.npy')
            UnMeasuredIdxsFull=np.load(SavePath + 'UnMeasuredIdxs.npy')    
            Difference = np.zeros(reconPercVector.shape[0])
            idx=0
            
            for j in tqdm(range(0, len(reconPercVector)), desc = '% Reconstructed', leave = True): #For each of the reconstruction percentages
                p = reconPercVector[j]
                NumMeasurements = int(p*SizeImage[0]*SizeImage[1]/100)
                MeasuredValues = MeasuredValuesFull[0:NumMeasurements]
                MeasuredIdxs = MeasuredIdxsFull[0:NumMeasurements][:]
                temp1 = MeasuredIdxsFull[NumMeasurements+1:MeasuredValuesFull.shape[0]][:]
                temp2 = UnMeasuredIdxsFull
                UnMeasuredIdxs = np.concatenate((temp1, temp2), axis=0)
                Difference[idx], ReconImage = performReconOnce(TrainingInfoObject,Resolution,SizeImage,ImageType,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,img)
                idx = idx+1
            
            TD = Difference/(SizeImage[0]*SizeImage[1])
            MSE = (np.sum((ReconImage.astype("float") - img.astype("float")) ** 2))/(float(ReconImage.shape[0] * ReconImage.shape[1]))
            SSIM = compare_ssim(ReconImage.astype("float"), img.astype("float"))
            np.save(SavePath + 'TD', TD)
            np.save(SavePath + 'MSE', MSE)
            np.save(SavePath + 'SSIM', SSIM)
            AreaUnderCurve[Idx_c]=AreaUnderCurve[Idx_c]+np.trapz(TD,x=reconPercVector)
        
        Idx_c = Idx_c +1
        
    Best_c = c_vec[np.argmin(AreaUnderCurve)]
    sys.path.pop() #Hopefully remove the 'sys.path.append('code') flag at the top of this definition
    return Best_c, NumImages

def runSLADS(InitialMaskObject,Mask,codePath,ImageSet,SizeImage,StopCondParamsObject, \
             Theta,TrainingInfoObject,TestingInfoObject,Resolution,ImageType,UpdateERDParamsObject, \
             BatchSamplingParamsObject,dataPath,resultsPath,fileName,ImNum,ImageExtension,Classify,isRunningParallel, \
             lineScanning,img,testingFlag,animationFlag,simulationFlag,trainingPlotFlag):
    sys.path.append('code')
    MeasuredIdxs = np.transpose(np.where(Mask==1))
    UnMeasuredIdxs = np.transpose(np.where(Mask==0))

    
    ContinuousMeasuredValues = performInitialMeasurements(Mask,simulationFlag,img)
    InitialMaskObject.delPoints(InitialMaskObject.initialMaskPts)
    if (len(ContinuousMeasuredValues) == 0):
        sys.exit('Error!!! No values were returned from initial measurements.')
    if Classify=='2C':
        Threshold = filters.threshold_otsu(ContinuousMeasuredValues)
        print('Threhold found using the Otsu method for 2 Class classification = ' + str(Threshold))
        MeasuredValues = ContinuousMeasuredValues < Threshold
        MeasuredValues = MeasuredValues+0
#    elif Classify=='MC':
        #### Classification function to output NewValues ##################
        # NewValues is the vector of measured values post classification
    elif Classify=='N':
        MeasuredValues=ContinuousMeasuredValues

    # Perform SLADS
    IterNum=1
    StopCondFuncVal=np.zeros(( int((SizeImage[0]*SizeImage[1])*(StopCondParamsObject.MaxPercentage)/100)+10,2 ))
    pixelCount = SizeImage[0]*SizeImage[1]
    percMeasured = 0
    completedRunFlag = False
    resultsObject = SLADSResults() 
    timesToDetermine = []
    noPointsFlag = False
    
    if (simulationFlag and testingFlag) or trainingPlotFlag:
        resultsObject.imgInit(img)
    
    with tqdm(total = 100, desc = '% Sampled', leave = True, disable = isRunningParallel) as pbar:
        while not completedRunFlag:
            if IterNum == 1:
                NumSamples = np.shape(MeasuredValues)[0]
                t0 = time.time()
                InitialMaskObject,Mask,MeasuredValues,ERDValues,ERDValuesNP,Difference,ReconValues,ReconImage,NewIdxs,MaxIdxsVect=firstERDandFindNewLocation(num_threads,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,UpdateERDParamsObject,BatchSamplingParamsObject,StopCondFuncVal,IterNum,NumSamples,None,None,None,None,None,None,lineScanning,InitialMaskObject,img,None)
                timesToDetermine.append(time.time()-t0)
                NumSamples = np.shape(MeasuredValues)[0]

                #Check to see if this completes the stop conditions, then save data according to simulation/animation conditions
                if checkStopCondFuncThreshold(StopCondParamsObject,StopCondFuncVal,NumSamples,IterNum,SizeImage) == 1:
                    completedRunFlag = True
                resultsObject.saveData(resultsPath, fileName, percMeasured, NumSamples, pixelCount, SizeImage, Mask, ReconImage, ERDValuesNP, ImageType, animationFlag, simulationFlag, testingFlag, completedRunFlag, noPointsFlag, trainingPlotFlag)
            else:
                t0 = time.time()
                InitialMaskObject,Mask,MeasuredValues,ERDValues,ERDValuesNP,Difference,ReconValues,ReconImage,NewIdxs,MaxIdxsVect=firstERDandFindNewLocation(num_threads,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,UpdateERDParamsObject,BatchSamplingParamsObject,StopCondFuncVal,IterNum,NumSamples,NewIdxs,ReconValues,ReconImage,ERDValues,ERDValuesNP,MaxIdxsVect,lineScanning,InitialMaskObject,img,Difference)
                timesToDetermine.append(time.time()-t0)
            if completedRunFlag:
                break
            if (len(NewIdxs) <= 0):
                completedRunFlag=True
                noPointsFlag = True
                resultsObject.saveData(resultsPath, fileName, percMeasured, NumSamples, pixelCount, SizeImage, Mask, ReconImage, ERDValuesNP, ImageType, animationFlag, simulationFlag, testingFlag, completedRunFlag, noPointsFlag, trainingPlotFlag)
                break
                
            #Perform measurements; Modify performMeasurements and performInitialMeasurements for non-simulation
            NewContinuousValues = performMeasurements(NewIdxs,BatchSamplingParamsObject,simulationFlag,img)
            ContinuousMeasuredValues = np.hstack((ContinuousMeasuredValues,NewContinuousValues))
            if Classify=='2C':           
                NewValues = NewContinuousValues > Threshold
                NewValues = NewValues+0
             #elif Classify=='MC':
             #   #### Classification function to output NewValues ##################
             #   # NewValues is the vector of measured values post classification            
            elif Classify=='N':
                NewValues=NewContinuousValues

            #Make a duplicate of the ReconValues for gradient test in computeStopCondFuncVal
            oldReconValues = ReconValues.copy()
            
            #Update measurement/mask/ERD information prior to the next loop or exiting
            NumSamples,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,ERDValues,ERDValuesNP,ReconValues,Difference,ReconImage = updateMeasurementArrays(NewIdxs,MaxIdxsVect,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,NewValues,BatchSamplingParamsObject,SizeImage,lineScanning,Theta,TrainingInfoObject,Resolution,ImageType,UpdateERDParamsObject,ReconValues,ReconImage,ERDValues,img)
            
            #Evaluate the stop condition value 
            StopCondFuncVal = computeStopCondFuncVal(oldReconValues,MeasuredValues,StopCondParamsObject,ImageType,StopCondFuncVal,MaxIdxsVect,NumSamples,IterNum,BatchSamplingParamsObject,lineScanning,SizeImage)
            
            #Check to see if the measurements complete the run
            if checkStopCondFuncThreshold(StopCondParamsObject,StopCondFuncVal,NumSamples,IterNum,SizeImage) == 1:
                completedRunFlag = True
            
            #Save data according to simulation/animation conditions
            resultsObject.saveData(resultsPath, fileName, percMeasured, NumSamples, pixelCount, SizeImage, Mask, ReconImage, ERDValuesNP, ImageType, animationFlag, simulationFlag, testingFlag, completedRunFlag, noPointsFlag, trainingPlotFlag)
            
            #Update percent measured information
            percMeasured = (NumSamples/pixelCount)*100
            IterNum += 1
            pbar.n = round(percMeasured)
            pbar.refresh()
            
    #Save final value information in .mat files
    np.save(dataPath + 'MeasuredValues', MeasuredValues)
    np.save(dataPath + 'MeasuredIdxs', MeasuredIdxs)
    np.save(dataPath + 'UnMeasuredIdxs', UnMeasuredIdxs)
    np.save(dataPath + 'StopCondFuncVal',StopCondFuncVal)
    np.save(dataPath + 'ContinuousMeasuredValues',ContinuousMeasuredValues)
    savemat(dataPath + 'MeasuredIdxs.mat',dict(MeasuredIdxs=MeasuredIdxs))
    savemat(dataPath + 'MeasuredValues.mat',dict(MeasuredValues=MeasuredValues))
    savemat(dataPath + 'UnMeasuredIdxs.mat',dict(UnMeasuredIdxs=UnMeasuredIdxs))
    savemat(dataPath + 'StopCondFuncVal.mat',dict(StopCondFuncVal=StopCondFuncVal))
    savemat(dataPath + 'ContinuousMeasuredValues.mat',dict(ContinuousMeasuredValues=ContinuousMeasuredValues))
    
    sys.path.pop() #Hopefully remove the 'sys.path.append('code') flag at the top of this definition
    
    #Save the times needed to acquire new locations each step
    resultsObject.timesToDetermine = timesToDetermine
    
    return resultsObject

def performInitialMeasurements(Mask,simulationFlag,img):
    if simulationFlag:
        if Mask.shape[0]!=img.shape[0] or Mask.shape[1]!=img.shape[1]:
            sys.exit('Error!!! The dimensions you entered in "SizeImage" do not match the dimensions of the testing image in ./ResultsAndData/TestingImages/TestingImageSet_' + ImageSet)
        MeasuredValues = img[Mask==1]
    else: #The user is expected to incorporate code here to harvest the data from an external source
        ##################################################################
        # INSTRUMENT CODE HERE
        # Plug in Your Measurement Routine
        # NewContinuousValues = performMeasurements(NewIdxs)
        ##################################################################   
        sys.exit('Error!!! - No external measurement code has been added to perform actual measurements!')
    return(MeasuredValues)

def performMeasurements(NewIdxs,BatchSamplingParamsObject,simulationFlag,img):
    if simulationFlag:
        NewValues = img[NewIdxs[:,0],NewIdxs[:,1]]
    #else: #The user is expected to incorporate code here to harvest the data from an external source
        ##################################################################
        # INSTRUMENT CODE HERE
        # Plug in Your Measurement Routine
        # NewContinuousValues = performMeasurements(NewIdxs)
        ##################################################################   
    return NewValues

def updateMeasurementArrays(NewIdxs,MaxIdxsVect,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,NewValues,BatchSamplingParamsObject,SizeImage,lineScanning,Theta,TrainingInfoObject,Resolution,ImageType,UpdateERDParamsObject,ReconValues,ReconImage,ERDValues,img):
    if not lineScanning:
        for i in range(0,len(NewIdxs)):
            Mask[NewIdxs[i,0],NewIdxs[i,1]]=1
    else: #lineScanning
        for pt in NewIdxs:
            Mask[tuple(pt)] = 1
    MeasuredValues = np.hstack((MeasuredValues,NewValues))
    MeasuredIdxs = np.vstack((MeasuredIdxs,NewIdxs))
    UnMeasuredIdxs = np.delete(UnMeasuredIdxs,(MaxIdxsVect), axis=0)
    
    
    NumSamples = np.shape(MeasuredValues)[0]
    ERDValues,ReconValues = updateERD(Mask,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,NewIdxs,NumSamples,UpdateERDParamsObject,ReconValues,ReconImage,ERDValues,MaxIdxsVect,BatchSamplingParamsObject)
    Difference,ReconImage = performReconOnce(TrainingInfoObject,Resolution,SizeImage,ImageType,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,img)
    ERDValuesNP = MakeERDArray(ERDValues, SizeImage, UnMeasuredIdxs, ReconImage)
    
    return(NumSamples,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,ERDValues,ERDValuesNP,ReconValues,Difference,ReconImage)

def findNewMeasurementIdxs(num_threads, Mask,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,NumSamples,UpdateERDParamsObject,ReconValues,ReconImage,ERDValues,BatchSamplingParams,lineScanning,InitialMaskObject):
    NewIdxs = []
    MaxIdxsVect = []
    if not lineScanning:
        for i in range(0,BatchSamplingParams.NumSamplesPerIter): #For the number of pixels to sample
            MaxIdxValue = np.nanargmax(ERDValues) #Choose the maximal ERD pixel
            ERDValues[MaxIdxValue] = np.nan #Remove the value from future consideration
            NewIdxs.append(UnMeasuredIdxs[MaxIdxValue].tolist()) #Add the positions to scan to the list
            MaxIdxsVect.append(MaxIdxValue)
    else: #lineScanning
        ERDValuesNP = MakeERDArray(ERDValues, SizeImage, UnMeasuredIdxs, ReconImage)
        if InitialMaskObject.MaskType == 'U': #Uniform random random full linescanning
            lineToScanIdx = random.randint(0, len(InitialMaskObject.linesToScan)-1)
            for i in range(0,SizeImage[1]): 
                NewIdxs.append([lineToScanIdx, i])
        elif InitialMaskObject.MaskType == 'H': #Low-discrepency random full linescanning
            lineNumber, seed = i4_uniform(0, len(InitialMaskObject.linesToScan)-1, seed)
            lineToScanIdx = int(lineNumber)
            for i in range(0,SizeImage[1]): 
                NewIdxs.append([lineToScanIdx, i])
        elif not InitialMaskObject.MaskType == 'M': #Full 0 deg lineScanning
            if InitialMaskObject.useERDmean:
                lineToScanIdx = np.nanargmax(np.nanmean(ERDValuesNP,axis=1))
            else:
                lineToScanIdx = np.nanargmax(np.nansum(ERDValuesNP,axis=1))
            for i in range(0,SizeImage[1]): 
                NewIdxs.append([lineToScanIdx, i])
        else: #Manual angle selected
            lineERDValues = []
            for line in InitialMaskObject.linesToScan:
                if InitialMaskObject.useERDmean:
                    lineERDValues.append(np.nanmean(ERDValuesNP[tuple([x[0] for x in line]), tuple([y[1] for y in line])]))
                else:
                    lineERDValues.append(np.nansum(ERDValuesNP[tuple([x[0] for x in line]), tuple([y[1] for y in line])]))
            
            #Select the line with maximal value
            lineToScanIdx = np.nanargmax(lineERDValues)
            lineERDValues[lineToScanIdx]
            
            startIDXFound = False
            endIDXFound = False
            if InitialMaskObject.partialLine and lineERDValues[lineToScanIdx] > 0:
                for idx in InitialMaskObject.linesToScan[lineToScanIdx]:
                    if ERDValuesNP[tuple(idx)] > 0 and not startIDXFound:
                        startIDX = idx
                        startIDXFound = True
                    if ERDValuesNP[tuple(idx)] > 0:
                        endIDX = idx
                #Add points between start and end idxs to the list if they were determiuned
                if (startIDXFound):
                    for idx in InitialMaskObject.linesToScan[lineToScanIdx]:
                        if idx[0] >= startIDX[0] and idx[1] >= startIDX[1] and idx[0] <= endIDX[0] and idx[1] <= endIDX[1]:
                            NewIdxs.append(idx)
                else:
                    Tracer()()
            else:
                NewIdxs = InitialMaskObject.linesToScan[lineToScanIdx]

            
            
            #The following conditional allows for partial line scanning
            #Dr. Ye has requested that this be disabled in favor of a more rough start/end point selection
            #This makes the assumption of a convex object
            #Scan only pixels in the line with non-zero ERD and reconMask Values
            #if InitialMaskObject.partialLine: 
            #    for idx in InitialMaskObject.linesToScan[lineToScanIdx]:
            #        if ERDValuesNP[tuple(idx)] > 0: 
            #            NewIdxs.append(idx)
            #else:
            #    NewIdxs = InitialMaskObject.linesToScan[lineToScanIdx]

            #If this procedure yields no new pixels...then just start scanning lines
            #if len(NewIdxs) <= 0:
            #    for line in InitialMaskObject.linesToScan:
            #        if len(line) > 0:
            #            NewIdxs = line
        #The following conditional allows for unmeasured points on a line that has already been scanned
        #to be considered
        #Dr. Yes has requested that this be disabled in favor of whole lines being removed from consideration
        #if InitialMaskObject.partialLine:
        #    InitialMaskObject.delPoints(NewIdxs)
        #else:
        #    InitialMaskObject.delLine(lineToScanIdx)
        
        InitialMaskObject.delLine(lineToScanIdx)
        
        for k in range(0, len(NewIdxs)):
            pt = np.asarray(NewIdxs).tolist()[k]
            if pt in UnMeasuredIdxs.tolist():
                MaxIdxsVect.append(UnMeasuredIdxs.tolist().index(pt))

    NewIdxs = np.asarray(NewIdxs) #Actual coordinates to measure next
    MaxIdxsVect = np.asarray(MaxIdxsVect) #Positions of those coordinates within the unmeasured listing

    return(NewIdxs,MaxIdxsVect)

def loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning):
    if lineScanning: #Need to reset the remaining linesToScan
        InitialMaskObject.linesToScan = InitialMaskObject.originalLinesToScan.copy()
    StartingMeasurementMask=InitialMaskObject.MaskType + '_' + str(InitialMaskObject.MaskNumber) + '_' + str(InitialMaskObject.RowSz) + '_' + str(InitialMaskObject.ColSz) + '_Percentage_' + str(InitialMaskObject.Percentage) + '_lineScanning_' + str(lineScanning)
    loadPathInitialMask = loadPathInitialMask + os.path.sep + StartingMeasurementMask
    if not os.path.exists(loadPathInitialMask):
        os.mkdir(loadPathInitialMask)
        Mask = generateInitialMask(InitialMaskObject,SizeImage,lineScanning)
        np.save(loadPathInitialMask + os.path.sep + 'InitialMask.npy', Mask)
    Mask = np.load(loadPathInitialMask + os.path.sep + 'InitialMask.npy')  
    return Mask

def computeStopCondFuncVal(ReconValues,MeasuredValues,StopCondParamsObject,ImageType,StopCondFuncVal,MaxIdxsVect,NumSamples,IterNum,BatchSamplingParamsObject,lineScanning,SizeImage):
    
    Diff = 0
    #Calculate the difference in values between the previous reconstruction values against the measured values
    for i in range(0, len(MaxIdxsVect)):
        a = ReconValues[MaxIdxsVect[i]]
        b = MeasuredValues[len(MeasuredValues)-len(MaxIdxsVect)+i]
        Diff = computeDifference(a,b,ImageType)+Diff
    Diff = Diff/len(MeasuredValues)
    if IterNum == 1:
        StopCondFuncVal[IterNum,0] = StopCondParamsObject.Beta*Diff
    else:
        StopCondFuncVal[IterNum,0] = ((1-StopCondParamsObject.Beta)*StopCondFuncVal[IterNum-1,0] + StopCondParamsObject.Beta*Diff)
    StopCondFuncVal[IterNum,1] = NumSamples
    return StopCondFuncVal

def checkStopCondFuncThreshold(StopCondParamsObject,StopCondFuncVal,NumSamples,IterNum,SizeImage):
    
    if StopCondParamsObject.Threshold==0:
        if NumSamples>=round(SizeImage[0]*SizeImage[1]*StopCondParamsObject.MaxPercentage/100):
            Stop=1
        else:
            Stop=0
    else:
        if NumSamples>=round(SizeImage[0]*SizeImage[1]*StopCondParamsObject.MaxPercentage/100):
            Stop=1
        else:
            if np.logical_and(((SizeImage[0]*SizeImage[1])*StopCondParamsObject.MinPercentage/100)<NumSamples,StopCondFuncVal[IterNum,0]<StopCondParamsObject.Threshold):
                Stop=0
                GradStopCondFunc =np.mean(StopCondFuncVal[IterNum,0]-StopCondFuncVal[IterNum-StopCondParamsObject.JforGradient:IterNum-1,0])
                if GradStopCondFunc<0:
                    Stop=1
            else:
                Stop=0
    return Stop

def computeBeta(SizeImage):
    if SizeImage[0]*SizeImage[1]<512**2+1:
        Beta = 0.001*(((18-math.log(SizeImage[0]*SizeImage[1],2))/2)+1)
    else:
        Beta = 0.001/(((math.log(SizeImage[0]*SizeImage[1],2)-18)/2)+1)
    return Beta     

def firstERDandFindNewLocation(num_threads,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,UpdateERDParamsObject,BatchSamplingParamsObject,StopCondFuncVal,IterNum,NumSamples,NewIdxs,ReconValues,ReconImage,ERDValues,ERDValuesNP,MaxIdxsVect,lineScanning,InitialMaskObject,img,Difference):
    
    #Only update the ERD if this is the first iteration; otherwise handled by updateMeasurementArray function
    if UpdateERDParamsObject.Do == 'N' or IterNum == 1:
        ERDValues,ReconValues,ReconImage = computeFullERD(MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType)
        Difference,ReconImage = performReconOnce(TrainingInfoObject,Resolution,SizeImage,ImageType,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,img)
        ERDValuesNP = MakeERDArray(ERDValues, SizeImage, UnMeasuredIdxs, ReconImage)
    NewIdxs,MaxIdxsVect = findNewMeasurementIdxs(num_threads,Mask,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,NumSamples,UpdateERDParamsObject,ReconValues,ReconImage,ERDValues,BatchSamplingParamsObject,lineScanning,InitialMaskObject)
    
    return(InitialMaskObject,Mask,MeasuredValues,ERDValues,ERDValuesNP,Difference,ReconValues,ReconImage,NewIdxs,MaxIdxsVect)

def performReconOnce(TrainingInfoObject,Resolution,SizeImage,ImageType,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,img):
    NeighborValues,NeighborWeights,NeighborDistances = FindNeighbors(TrainingInfoObject,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Resolution)
    ReconValues,ReconImage = ComputeRecons(TrainingInfoObject,NeighborValues,NeighborWeights,SizeImage,UnMeasuredIdxs,MeasuredIdxs,MeasuredValues)    
    Difference = np.sum(computeDifference(img,ReconImage,ImageType))
    return(Difference, ReconImage)

def findStoppingThreshold(trainingDataPath,NumTrainingImages,Best_c,Difference, ERDValuesNP, ReconValues, ReconImagePercentageInitialMask,DesiredTD,reconPercVector,SizeImage):
    sys.path.append('code')
    SavePathSLADS = trainingDataPath + 'SLADSResults' 
    Thresh = np.zeros(NumTrainingImages)
    count=0
    for ImNum in range(0,NumTrainingImages): 
        LoadPath = SavePathSLADS + os.path.sep + 'Image_' + str(ImNum+1) + '_c_'+ str(Best_c) + os.path.sep
        StopCondFuncVal = np.load(LoadPath + 'StopCondFuncVal.npy')
        TD = np.load(LoadPath + 'TD.npy')
        found=0
        for i in range(0,TD.shape[0]):
            if TD[i]<DesiredTD and found==0 :
                Idx = int((reconPercVector[i]-PercentageInitialMask)*SizeImage[0]*SizeImage[1]/100)
                Thresh[ImNum]=StopCondFuncVal[Idx][0]
                count=count+1
                found=1
    Threshold = np.sum(Thresh)/count
    sys.path.pop()
    return Threshold

def generateInitialMask(InitialMaskObject,SizeImage,lineScanning):
    Mask = np.zeros((SizeImage[0],SizeImage[1]))
    seed = int(((datetime.now()).microsecond))
    if not lineScanning:
        UnifMatrix = []
        if InitialMaskObject.MaskType == 'H':
            samplePoints = round((InitialMaskObject.Percentage/100)*(SizeImage[0]*SizeImage[1]))
            for i in range(0,samplePoints):
                row, seed = i4_uniform(0, SizeImage[1]-1, seed)
                column, seed = i4_uniform(0, SizeImage[0]-1, seed)
                Mask[row, column] = 1
        elif InitialMaskObject.MaskType == 'U':
            UnifMatrix = np.random.rand(SizeImage[0],SizeImage[1])
            Mask = UnifMatrix<(InitialMaskObject.Percentage/100)
        else:
            sys.exit('Error!!! - undefined handeling for the generation of the MaskType indicated')
    elif InitialMaskObject.MaskType == 'S': #If performing a spiral as the initial masking operation
        sys.exit('ERRROR!!! - Initial spiral masking not implemented yet')
    elif lineScanning:
        if not InitialMaskObject.MaskType == 'M': #If not using a manual specification
            linesScanned = []
            for i in range(0,round(SizeImage[0]*(InitialMaskObject.Percentage/100))):
                notFound = True
                while notFound:
                    if InitialMaskObject.MaskType == 'H':
                        lineNumber, seed = i4_uniform(0, SizeImage[0]-1, seed)
                        lineNumber = int(lineNumber)
                    elif InitialMaskObject.MaskType == 'U':
                        lineNumber = random.randint(0,SizeImage[0]-1)
                    else:
                        sys.exit('Error!!! - undefined handeling for the generation of the MaskType indicated')
                    if lineNumber not in linesScanned:
                        notFound = False
                        linesScanned.append(lineNumber)
            Mask[linesScanned] = 1
        else: #Using a manual angle specification, use the specfied points to inform the mask
            line = InitialMaskObject.initialMaskPts
            for pt in line:
                Mask[tuple(pt)] = 1
            InitialMaskObject.delPoints(InitialMaskObject.initialMaskPts)
    else:
        sys.exit('Error!!! - line scanning bypassed in generate initial mask.')
    Mask = Mask==1
    return Mask
        
def plotImage(Image,Num):
    plt.imshow(Image)
    #pylab.show()

def plotAfterSLADS(Im1,Im2):
    plt.figure(1)                
    plt.subplot(121)          
    plt.imshow(Im1)
    plt.title('Sampled Mask')   
    plt.subplot(122)        
    plt.imshow(Im2)
    plt.title('Reconstructed Image')

def importImages(dirPath, inputExtension, SizeImage):
    if (inputExtension == ".tif") or (inputExtension == ".png"):
        dataFileNames = glob.glob(dirPath + "/*" + inputExtension) #Obtain filenames for each set
        zLen = len(dataFileNames) #Find total number of files imported
        if (zLen == 0):
            sys.exit("Error!!! There are no files with extension: " + inputExtension + " in the directory: " + dirPath)
        dataset = Image.open(dataFileNames[0]).convert('L') #Read in an image as grayscale
        dataset = np.asarray(dataset,dtype=np.float64).reshape((dataset.size[1],dataset.size[0])) #Flatten the image
        #firstX, firstY = dataset.shape[0], dataset.shape[1] #Obtain the set's dimensions
        firstX, firstY = SizeImage[0], SizeImage[1]
        datasets = [] #Create an empty array to hold instances of imageData objects
        counter = 0
        for file in dataFileNames: #For each of the filenames
            dataset = Image.open(file).convert('L') #Read in an image as grayscale
            datasetXLen, datasetYLen = dataset.size #Obtain the set's dimensions
            if ((SizeImage[0], SizeImage[1]) != (datasetXLen, datasetYLen)): #If any of the images differ in dimensions
                if(debugInfo): print("Warning!!! File:", dataFileNames[counter], "has dimensions X: ", datasetXLen, " Y: ", datasetYLen, " - Will resize") #Inform the user
                dataset = dataset.resize([SizeImage[0], SizeImage[1]]) #Resize the file to the dimensions specified
            dataset = np.asarray(dataset,dtype=np.float64).reshape((dataset.size[1],dataset.size[0])) #Flatten the image
            outputName = file
            outputName = outputName[outputName.startswith(dirPath) and len(dirPath):]
            if inputExtension == ".tif":
                outputName = re.sub('\.tif$', '', outputName)
            elif inputExtension == ".png":
                outputName = re.sub('\.png$', '', outputName)
            datasets.append(imageData(dataset, outputName))
            counter+=1
    return datasets

#Sort values list by key list
def resultsSort(values,key):
    paired = []
    for i in range(0,len(key)): paired.append(tuple([key[i], values[i]]))
    paired = np.asarray(paired) 
    paired = paired[paired[:,1].argsort()[::1]]
    return paired[:,1].tolist()

def resultsAverager(testingResults):
    c = []
    for result in testingResults: c.append(result)
    meanValues = []
    stdValues = []
    filteredValues = []
    for x in zip_longest(*c): 
        filtered = []
        for value in x: 
            if value != None: 
                filtered.append(value)
        filteredValues.append(filtered)
        meanValues.append(np.mean(filtered))
        stdValues.append(np.std(filtered))
    return meanValues, stdValues

def perc_Results(results, perc_testingResults, precision):
    percents = np.arange(min(np.hstack(perc_testingResults)),max(np.hstack(perc_testingResults))+precision, precision)
    averages = []
    finalPercents = []
    for percent in percents:
        values = []
        for i in range(0,len(results)):
            percList = np.array(perc_testingResults[i])
            idx = np.argmin(np.abs(np.asarray(percList)-percent))
            values.append(results[i][idx])
        averageValue = np.mean(values)
        if len(averages) == 0 or averageValue != averages[len(averages)-1]:
            averages.append(np.mean(values))
            finalPercents.append(percent)
    return finalPercents, averages

def gausKern_parhelper(sigmaVal, WindowSize, area): #Parallel loop for generating a Gaussian kernel
    return area*generateGaussianKernel(sigmaVal,WindowSize) #Calculate an "area" that the c value will capture based on a gaussian filter

def stats_parhelper(InitialMaskObject, Mask, codePath, ImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, dataPath, resultsPath, fileName, ImNum, ImageExtension, Classify, isRunningParallel, lineScanning, img, testingFlag, animationFlag, simulationFlag, trainingPlotFlag):
    result = runSLADS(InitialMaskObject, Mask, codePath, ImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, dataPath, resultsPath, fileName, ImNum, ImageExtension, Classify, isRunningParallel, lineScanning, img, testingFlag, animationFlag, simulationFlag, trainingPlotFlag)
    return result

def cls(): #Clear console screen
    os.system('cls' if os.name=='nt' else 'clear')   


In [5]:
#MAIN PROGRAM
#==================================================================
cls() #Clear the screen
consoleRunning = False #Is the program being run in a console (alternative being jupyter-notebook)

#GENERAL PARAMETERS: L-01
#==================================================================

#Is training of a model to be performed
trainingModel = False

#Is testing of a model to be performed
testingModel = True

#Is the program a simulation, or actual implementation
simulationFlag = True

#Should full lines be selected for scanning
lineScanning = True

#For manual angle lineScanning 
#What should the angle spacing be for considered angles (manualAngleDiff must be > 0)
manualAngleDiff = 20
manualStartAngle = 0
#manualStopAngle = 180-manualAngleDiff
manualStopAngle = 0

#What points should be used for the initial mask if lineScanning
#C: The line that passes as close as possible to the middle of the image
#S: A spiral pattern
#M: A custom set of points
#MA: A manual angle passing through the center
#SG: Sparse Grid
initialMaskSelection = 'MA'

#If using 'MA' mask selection, specify the desired angle
manualInitialAngle = 0

#If using a custom set of points for the initialMaskSelection, specify
initialMaskCustom = [[0,0],[1,1]]

#If using a spiral pattern specify the number of turns
spiralTurns = 3

#If not lineScanning, how many pixels should be sampled at each step
#If lineScanning, how many lines should be sampled at each step (NOT IMPLEMENTED; LEAVE AS 1 FOR LINE SCANNING) 
batchSampleSize = 1

#Should the initial masks be overwritten (Change to false for comparative analysis with previous run)
initialMaskOverwrite = True

#Should the default multithreading be limited
overrideThreads = False

#If the default multithreading be limited how many threads should be used
num_threads = 1

#Should warning/debug information be displayed?
debugInfo = False

#Type of Image: D - for discrete (binary) image; C - for continuous
ImageType = 'C'

#What is the file extenstion of the data?
inputExtension = ".tif";

#What is the symmetric length all images should be resized to?
#(64x64), (128x128), (256x256), (512x512)
SizeImage = [64, 64]

#Find threshold for stopping function Y/N; If 'Y', set the DesiredTD in L-1
FindStopThresh = 'N'

#TRAINING MODEL PARAMETERS: L-02
#==================================================================

#Enter 'X' for TrainingInputDB_X
TrainingImageSet = '1'

#Should the input data be split into training/testing sets
#If true and customTesting is also true, then custom set will be added to the split of the trainingImageSet database
#If false and testingModel is true, then only custom set will be used
splitInputSet = False

#Should progressive testing data be plotted for the training data
trainingErrorPlot = True

#Should c value vs distortion metric be plotted; (NOT IMPLEMENTED)
cPlot = False

#Stopping percentage for SLADS (to select C)
#Suggested: (64x64):50, (128x128):30, (256x256):20, (512x512):10
StopPercentageSLADS =  50

#TESTING MODEL PARAMETERS: L-03
#==================================================================

#Is there a custom testing set of images to be used?
customTesting = True

#If customTesting set is true, enter 'X' for TestingInputDB_X
TestingImageSet = '1'

#If customTesting set is false, indicate % of data to use for training; test is 1-(trainSplit/100)
trainSplit = 80 

#Should error data be plotted for the testing data
testingErrorPlot = True

#Should the best c found during training be overidden?
overrideBestC = False

#If the best c should be overiden with a manual value what should that value be?
overrideBestCValue = 2

#Stopping percentages for SLADS during testing
StopPercentageTestingSLADS = 50
#PROGRAM PARAMETERS: L-1
#==================================================================

# Sweep range for c (to select best c for RD approximation)
#c_vec = np.asarray([2])
c_vec = np.array([2,4,8,16,32])

# Sampling mask measurement percentages for training (best left unchanged)
MeasurementPercentageVector = np.array([5, 10, 20, 30, 40, 50])
#MeasurementPercentageVector = np.array([5, 10, 20])

# Window size for approximate RD summation (best left unchanged)
#WindowSize = [15,15] #NOTE - WAS FOR 512x512 images!!!
WindowSize = [3,3]

# Update ERD or compute full ERD in SLADS (to find best c)
# with Update ERD, ERD only updated for a window surrounding new measurement
Update_ERD = 'Y' 

#If linescanning shoud the mean of the ERD values in the line, or their sum be used to max the min
useERDmean = False

#If linescanning should only the points predicted to be greater than background noise be scanned
partialLine = True

# Smallest ERD update window size permitted - NOT USED!!!
MinWindSize = 3  

# Largest ERD update window size permitted - NOT USED!!!
MaxWindSize = 10       

# Initial Mask for SLADS (to find best c):
# Percentage of samples in initial mask
PercentageInitialMask = 1

# Type of initial mask   
# Choices: 
    # 'U': Random Uniform mask; can choose any percentage
    # 'H': low-discrepancy mask; can choose any percentage
    # 'S': Spiral out from center; not implemented
    # 'M': Manual lineScanning angle; can choose any percentage
MaskType = 'M'

# Desired total distortion (TD) value (to find threshold on stopping function)
# TD = D(X,\hat(X))/(Number of pixels in image)
# D(X,\hat(X)) is difference between actual image X and reconstructed image 
# \hat(X) (summed over all pixels)
# For ImageType 'D' in range [0-1] for ImageType 'C' in range [0-max value]
DesiredTD=0


In [6]:
#STATIC VARIABLE SETUP
#==================================================================

#What is the file extention desired for the pre-processed data
ImageExtension = ".png"

#Set the number of cpu threads to be used for generating Gaussian Kernals
if not overrideThreads:
    num_threads = multiprocessing.cpu_count()-2 #Determine number of available threads, leave a couple as spares

#Determine console size
if consoleRunning:
    consoleRows, consoleColumns = os.popen('stty size', 'r').read().split()
else:
    consoleRows = 40
    consoleColumns = 40

#Set global plot parameters
plt.rcParams["font.family"] = "Times New Roman"
    
#Convert types as needed
c_vec = c_vec.astype(float)
MeasurementPercentageVector = MeasurementPercentageVector.astype(float)
StopPercentageSLADS = float(StopPercentageSLADS)
PercentageInitialMask = float(PercentageInitialMask)

NumReconsSLADS = 10 #UNKNOWN VARIABLE
PercOfRD = 50 #UNKNOWN VARIABLE
Classify = 'N' #UNKNOWN VARIABLE
Resolution = 1 #UNKNOWN VARIABLE

falseFlag = False #Hold a False variable for function calls to disable certain parameters

BatchSamplingParamsObject = BatchSamplingParams()
BatchSamplingParamsObject.initialize(batchSampleSize)

#Store training information as object
TrainingInfoObject = TrainingInfo()

#Initalize the training information as set through specified variables
if ImageType == 'D':
    TrainingInfoObject.initialize('DWM','DWM',2,10,'Gaussian',0,0.25,15)
elif ImageType == 'C':
    TrainingInfoObject.initialize('CWM','CWM',2,10,'Gaussian',0,0.25,15)

#Store testing information as object
TestingInfoObject = TestingInfo()
TestingInfoObject.initialize()

if lineScanning:
    manualAngles = np.linspace(manualStartAngle,manualStopAngle, int(((manualStopAngle-manualStartAngle)/manualAngleDiff)+1)).astype(int)
    linesToScan = allLinesForAngle(manualAngles)
    #Make a backup copy of the lines to scan 
    originalLinesToScan = linesToScan.copy()

    #Select the initial line to scan
    if initialMaskSelection == 'C':
        #Find the line that is closest to the center for the initial scan location
        midIndex = 0
        for i in range(0,len(linesToScan)):
            a = np.average(np.asarray(linesToScan[i])[:,0]) - (SizeImage[0]/2)
            b = np.average(np.asarray(linesToScan[i])[:,1]) - (SizeImage[1]/2)
            d = math.sqrt((a**2)+(b**2))
            if i == 0: minD = d
            if d < minD:
                minD = d
                midIndex = i
        initialMaskPts = linesToScan[midIndex]
    elif initialMaskSelection == 'MA':
        manualAngles = np.asarray([manualInitialAngle])
        linesToScan = allLinesForAngle(manualAngles)
        #Find the line that is closest to the center for the initial scan location
        midIndex = 0
        for i in range(0,len(linesToScan)):
            a = np.average(np.asarray(linesToScan[i])[:,0]) - (SizeImage[0]/2)
            b = np.average(np.asarray(linesToScan[i])[:,1]) - (SizeImage[1]/2)
            d = math.sqrt((a**2)+(b**2))
            if i == 0: minD = d
            if d < minD:
                minD = d
                midIndex = i
        initialMaskPts = []
        for pt in linesToScan[midIndex]: initialMaskPts.append(pt)
    elif initialMaskSelection == 'SG':
        initialMaskPts = []
        mid = int((SizeImage[1]/2)-1)
        xinterval = (SizeImage[0]-1)/int(SizeImage[0]/4)
        yinterval = (SizeImage[1]-1)/2

        if yinterval%2 == 0:
            ypos = yinterval-1
        else:
            ypos = 0
        while ypos < SizeImage[1]:
            if interval%2 == 0:
                xpos = interval-1
            else:
                xpos = 0
            while xpos < SizeImage[0]:
                initialMaskPts.append([int(ypos), int(xpos)])
                xpos+=xinterval
            ypos+=yinterval
    else:
        sys.exit('Error!!! - initialMaskSelection criteria specified has not yet been implemented.')

else:
    initialMaskPts = []
    originalLinesToScan = []
    linesToScan = []

InitialMaskObject = InitialMask()
InitialMaskObject.initialize(SizeImage[0],SizeImage[1],MaskType,1,PercentageInitialMask,originalLinesToScan,linesToScan,initialMaskPts,useERDmean,partialLine)

#PATH/DIRECTORY SETUP
#==================================================================

#Set starting data path
codePath = '.' + os.path.sep

#Set path to Results and Data Folder
resultsDataPath = codePath + 'ResultsAndData' + os.path.sep

#Check directory
if not os.path.exists(resultsDataPath):                                                                                                                          
    sys.exit('Error!!! The folder ' + resultsDataPath + ' does not exist. ')

#Clear initial sampling mask directory if overwrite enabled
if initialMaskOverwrite:
    loadPathInitialMask = resultsDataPath + 'InitialSamplingMasks'
    if os.path.exists(loadPathInitialMask):
            shutil.rmtree(loadPathInitialMask)
    os.makedirs(loadPathInitialMask)
else:
    StartingMeasurementMask=InitialMaskObject.MaskType + '_' + str(InitialMaskObject.MaskNumber) + '_' + str(InitialMaskObject.RowSz) + '_' + str(InitialMaskObject.ColSz) + '_Percentage_' + str(InitialMaskObject.Percentage) + '_lineScanning_' + str(lineScanning)
    loadPathInitialMask = loadPathInitialMask + os.path.sep + StartingMeasurementMask + os.path.sep + 'InitialMask.npy'
    if not os.path.exists(loadPathInitialMask):
        sys.exit('Error!!! - A suitable initial mask does not exist for the parameters given; enable initialMaskOverwrite to generate one')

TrainingDBName = 'TrainingDB_' + TrainingImageSet
    
#Path to the input training image database
dataPath = resultsDataPath + 'InputData' + os.path.sep + 'InputTrainingDB_' + str(TrainingImageSet) + os.path.sep
if not os.path.exists(dataPath):                                                                                                                          
    sys.exit('Error!!! The folder ' + dataPath + ' does not exist. Check entry for ' + TrainingImageSet)

        
#Path to where training resources/features should be saved
trainingFeaturesPath = resultsDataPath + 'TrainingSavedFeatures' + os.path.sep
    
if trainingModel: #If training is to be performed, check and clean the relevant directories
    
    #With regards to where training resources/features should be saved
    trainingFeaturesPath = resultsDataPath + 'TrainingSavedFeatures' + os.path.sep
    if os.path.exists(trainingFeaturesPath):
        shutil.rmtree(trainingFeaturesPath)
    os.makedirs(trainingFeaturesPath)
          
    if trainingErrorPlot:
        trainingPlotFeaturesPath = trainingFeaturesPath + 'Plot' + os.path.sep
        if os.path.exists(trainingPlotFeaturesPath):
            shutil.rmtree(trainingPlotFeaturesPath)
        os.makedirs(trainingPlotFeaturesPath)
    
    #Set path to where training data should be kept
    trainingDataPath = trainingFeaturesPath + 'TrainingDB_' + str(TrainingImageSet) + os.path.sep
    if not os.path.exists(trainingDataPath):                                                                                                                          
        os.makedirs(trainingDataPath)
    
    #Set path to where training data images should be kept
    trainingDataImagesPath = trainingDataPath + 'Images' + os.path.sep   
    if os.path.exists(trainingDataImagesPath):
        shutil.rmtree(trainingDataImagesPath)
    os.makedirs(trainingDataImagesPath)
        
    #Clear and setup training model path
    if os.path.exists(trainingDataPath + 'FeaturesRegressCoeffs'):
        shutil.rmtree(trainingDataPath + 'FeaturesRegressCoeffs')
    os.makedirs(trainingDataPath + 'FeaturesRegressCoeffs')
    
if testingModel: #If testing is to be performed, check the relevant directories

    #Path to where training resources/features should be saved
    testingFeaturesPath = resultsDataPath + 'TestingSavedFeatures' + os.path.sep
    if os.path.exists(testingFeaturesPath):
        shutil.rmtree(testingFeaturesPath)
    os.makedirs(testingFeaturesPath)
        
    #Set path to where testing data should be kept
    testingDataPath = testingFeaturesPath + 'TestingDB_' + str(TrainingImageSet) + os.path.sep
    if not os.path.exists(testingDataPath):                                                                                                                          
        os.makedirs(testingDataPath)
        
    #Set path to where initial input files are located
    if (customTesting): #If a custom training set is going to be used
        customTestingDataPath = resultsDataPath + 'InputData' + os.path.sep + 'InputTestingDB_' + str(TestingImageSet) + os.path.sep
        if not os.path.exists(customTestingDataPath):
            sys.exit('Error!!! The folder ' + customTestingDataPath + ' does not exist. Check entry for ' + customTestingDataPath + ' or set the customTesting flag to false.')
    
    #Set data path to where data should be split from
    if not customTesting and not trainingModel and splitInputSet:
        dataPath = resultsDataPath + 'InputData' + os.path.sep + 'InputTrainingDB_' + str(TrainingImageSet) + os.path.sep
        
    #Set path to where testing data images should be kept 
    testingDataImagesPath = testingDataPath + 'Images' + os.path.sep   
    if os.path.exists(testingDataImagesPath):
        shutil.rmtree(testingDataImagesPath)
    os.makedirs(testingDataImagesPath)
        
    #Clear and setup testing model path
    if os.path.exists(testingDataPath + 'FeaturesRegressCoeffs'):
        shutil.rmtree(testingDataPath + 'FeaturesRegressCoeffs')
    os.makedirs(testingDataPath + 'FeaturesRegressCoeffs')

    #Set path to save training results
    TrainingSavePath = resultsDataPath + 'SLADSSimulationResults' + os.path.sep + 'TrainingDB_' + TrainingImageSet + os.path.sep
    if os.path.exists(TrainingSavePath):
        shutil.rmtree(TrainingSavePath)
    os.makedirs(TrainingSavePath)
    
    #Set path to save testing results
    testingSavePath = resultsDataPath + 'SLADSSimulationResults' + os.path.sep + 'TestingDB_' + TestingImageSet + os.path.sep
    if os.path.exists(testingSavePath):
        shutil.rmtree(testingSavePath)
    os.makedirs(testingSavePath)
    
    #Set path for saving the resulting images from the testing results
    testingImagesSavePath = testingSavePath + 'ImageResults' + os.path.sep    
    if os.path.exists(testingImagesSavePath):
        shutil.rmtree(testingImagesSavePath)  
    os.makedirs(testingImagesSavePath)

    AnimationImagesSavePath = testingImagesSavePath + 'Animation' + os.path.sep
    if os.path.exists(AnimationImagesSavePath): 
        shutil.rmtree(AnimationImagesSavePath)
    os.makedirs(AnimationImagesSavePath)
    
    #Set path for saving the resulting training statistics
    trainingStatisticsSavePath = TrainingSavePath + 'Statistics' + os.path.sep    
    if os.path.exists(trainingStatisticsSavePath):
        shutil.rmtree(trainingStatisticsSavePath)  
    os.makedirs(trainingStatisticsSavePath)    

    #Set path for saving the resulting testing statistics
    testingStatisticsSavePath = testingSavePath + 'Statistics' + os.path.sep    
    if os.path.exists(testingStatisticsSavePath):
        shutil.rmtree(testingStatisticsSavePath)  
    os.makedirs(testingStatisticsSavePath)    
    
    #Set the path for where the initial mask locations are 
    loadPathInitialMask = resultsDataPath + 'InitialSamplingMasks' #Set initial mask path
    if not os.path.exists(loadPathInitialMask):                                                                                                                          
        sys.exit('Error!!! The folder ' + loadPathInitialMask + ' does not exist. Check entry for ' + loadPathInitialMask)

In [7]:
#DATA IMPORTATION
#==================================================================
print('#' * int(consoleColumns))
print('PRE-PROCESSING DATA')
print('#' * int(consoleColumns) + '\n')
#Check validity of training/testing parameters
if testingModel and not splitInputSet and not customTesting:
    sys.exit("ERROR!!! Testing enabled, but neither custom, nor split input sets enabled")

if splitInputSet and not testingModel:
    if(debugInfo): print("WARNING!!! Testing disabled, but splitInputSet enabled. Only the specified fraction of the training set will be used")

#If any training is to be performed
if trainingModel: 
    datasets = importImages(dataPath, inputExtension, SizeImage) #Import images and perform pre-processing
    datasets = shuffle(datasets) #Randomize dataset order
    if splitInputSet: #If data should be split
        numTrain = int(len(datasets)*(trainSplit/100)) #Find number of training examples; round as int for indexing
        trainingData = datasets[0:numTrain] #Split off the training dataset
    else: #The data should not be split
        trainingData = datasets #Simply pass along the data as the training set
    
    #Save pre-processed training images   
    for i in range(0,len(trainingData)): #For each dataset in the training data
        dataset = trainingData[i]
        grayImage=Image.fromarray(dataset.data).convert("L") #Reset the values for the image such that they are visible
        grayImage.save(trainingDataImagesPath+dataset.name+ImageExtension) #Save the output image
        trainingData[i] = misc.imread(trainingDataImagesPath+dataset.name+ImageExtension)
if testingModel:
    if splitInputSet and not trainingModel:
        datasets = importImages(dataPath, inputExtension, SizeImage) #Import images and perform pre-processing
        datasets = shuffle(datasets) #Randomize dataset order
        numTrain = int(len(datasets)*(trainSplit/100)) #Find number of training examples; round as int for indexing
    
    #If only split data from the training set is to be used for testing
    if not customTesting and splitInputSet:
        testingData = datasets[numTrain:(len(datasets))]

    #If split data and custom data should be used for testing
    if customTesting and splitInputSet:
        testingData = datasets[numTrain:(len(datasets))] + importImages(customTestingDataPath, inputExtension, SizeImage)

    #If only custom data should be used for testing
    if customTesting and not splitInputSet:
        testingData = importImages(customTestingDataPath, inputExtension, SizeImage)
    
    #Save pre-processed testingData
    for i in range(0, len(testingData)): 
        dataset = testingData[i]
        grayImage=Image.fromarray(dataset.data).convert("L") #Reset the values for the image such that they are visible
        grayImage.save(testingDataImagesPath+dataset.name+ImageExtension) #Save the output image
        testingData[i] = misc.imread(testingDataImagesPath+dataset.name+ImageExtension) #Import into final testing dataset

########################################
PRE-PROCESSING DATA
########################################



In [ ]:
#RUN TRAINING
#==================================================================  
if trainingModel:
    print(' \r') #Reset cursor
    print('#' * int(consoleColumns))
    print('TRAINING MODEL')
    print('#' * int(consoleColumns) + '\n')
    
    NumTrainingImages = len(trainingData)
    imagePercent = (1/NumTrainingImages)*100
    isRunningParallel = True
    if trainingErrorPlot:
        testingImageFiles = glob.glob(testingDataImagesPath + "/*" + ImageExtension) #Obtain filenames for each set

    #Until the SLADS stopping percentage value
    #Vector of values to determine for the reconstruction
    reconPercVector = np.linspace(PercentageInitialMask, StopPercentageSLADS, num=NumReconsSLADS*(StopPercentageSLADS-PercentageInitialMask), endpoint=False)

    #For each of the Sampling mask measurement percentages for training
    #For each of the images want to maximize the reduction in distortion 
    #Train regression formula on known images with randomly selected sampling densities
    #Estimation of distortion reduction based on a provided weight parameter and the distance between the previous and proposed pixel location
    samplingPercent = (imagePercent/np.size(MeasurementPercentageVector))

    StopNumRows = StopPercentageSLADS*SizeImage[0]
    cPercent = samplingPercent/len(c_vec)

    for ImNum in tqdm(range(0,len(trainingData)), desc = 'Training Images', leave = True):
        Img = trainingData[ImNum] #Retreive a training image
        try:
            Img
        except NameError:
            sys.exit('Error!!! There are no images in the training set!')

        for m in tqdm(range(0,np.size(MeasurementPercentageVector)), desc = 'Sampling Densities', leave = consoleRunning): #For each of the proposed sampling densities
            #Create a save folder for the produced information
            SaveFolder = 'Image_' + str(ImNum+1) + '_SampPerc_' + str(MeasurementPercentageVector[m])
            trainingFeaturesSavePath = trainingDataPath + 'FeaturesRegressCoeffs' + os.path.sep + SaveFolder
            if not os.path.exists(trainingFeaturesSavePath):
                os.makedirs(trainingFeaturesSavePath)

            Mask = loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning)

            MeasuredIdxs = np.transpose(np.where(Mask==1))
            UnMeasuredIdxs = np.transpose(np.where(Mask==0))            
            MeasuredValues = Img[Mask==1]

            # Find neighbors
            NeighborValues,NeighborWeights,NeighborDistances = FindNeighbors(TrainingInfoObject,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,Resolution)

            #Form the reconstructed image
            ReconValues,ReconImage = ComputeRecons(TrainingInfoObject,NeighborValues,NeighborWeights,SizeImage,UnMeasuredIdxs,MeasuredIdxs,MeasuredValues)

            #Compute the image features
            AllPolyFeatures=computeFeatures(UnMeasuredIdxs,SizeImage,NeighborValues,NeighborWeights,NeighborDistances,TrainingInfoObject,ReconValues,ReconImage,Resolution,ImageType)
            
            #Determine a number of points  unmasked percent * percentage of distortion reduction * imageArea / (100*100)
            NumRandChoices = int((100-MeasurementPercentageVector[m])*PercOfRD*SizeImage[1]*SizeImage[0]/(100*100))
            
            #Create a sample based on a number of previously unmeasured points
            OrderForRD = random.sample(range(0,UnMeasuredIdxs.shape[0]), NumRandChoices) 

            #Extract the image features of interest from the randomly selected unmeasured points
            PolyFeatures = AllPolyFeatures[OrderForRD,:]
            
            #Use everything version
            #OrderForRD = np.linspace(0,len(UnMeasuredIdxs)-1,len(UnMeasuredIdxs)).astype(int)
            #PolyFeatures = AllPolyFeatures[OrderForRD,:]

            #Compute the differences between the original and reconstructed images
            RDPP = computeDifference(Img,ReconImage,ImageType)

            #Round differences to nearest integer
            RDPP.astype(int)

            #Pad the differences
            RDPPWithZeros = np.lib.pad(RDPP,(int(np.floor(WindowSize[0]/2)),int(np.floor(WindowSize[1]/2))),'constant',constant_values=0)

            #Convert image to an array

            #ImgAsBlocks = im2col(RDPPWithZeros,WindowSize)

            #Slighly faster implementation of im2col
            N = RDPPWithZeros.shape[1] - WindowSize[1] + 1
            M = RDPPWithZeros.shape[0] - WindowSize[0] + 1
            ImgAsBlocks = viewW(RDPPWithZeros, (M,N)).reshape(-1,M*N)[:,::1]

            
            #Flatten 2D mask array to 1-D
            MaskVect = np.ravel(Mask)

            #Identify the pixels that have not yet been measured
            ImgAsBlocksOnlyUnmeasured = ImgAsBlocks[:,np.logical_not(MaskVect)]
            temp = np.zeros((WindowSize[0]*WindowSize[1],len(OrderForRD)))
            for n in tqdm(range(0,len(c_vec)), desc = 'c Values', leave = consoleRunning): #For each of the possible c values
                c = c_vec[n]
                sigma = NeighborDistances[:,0]/c
                cnt = 0;
                
                #For each of the unmeasured points (limited by OrderForRD) calculate the area 
                area = []
                for index in tqdm(range(0,len(OrderForRD)), desc = 'Gaussian', leave = consoleRunning): #Perform task in parallel
                    result = gausKern_parhelper(sigma[OrderForRD[index]], WindowSize, ImgAsBlocksOnlyUnmeasured[:,OrderForRD[index]])
                    area.append(result)
                #area = Parallel(n_jobs=num_threads)(delayed(gausKern_parhelper)(sigma[OrderForRD[index]], WindowSize, ImgAsBlocksOnlyUnmeasured[:,OrderForRD[index]]) 
                
                for i in range (0,len(OrderForRD)): temp[:,i] = area[i]
                RD = np.sum(temp, axis=0) #Determine how much "area of uncertainty" is possibly removed for a c value
                
                #Save everything
                SavePath_c = trainingFeaturesSavePath + os.path.sep + 'c_' + str(c)
                if not os.path.exists(SavePath_c):
                    os.makedirs(SavePath_c)
                np.save(SavePath_c + os.path.sep + 'RD' + '_StopPerc_' + str(StopPercentageSLADS), RD)        
                np.save(SavePath_c + os.path.sep + 'OrderForRD' + '_StopPerc_' + str(StopPercentageSLADS), OrderForRD)

            np.save(trainingFeaturesSavePath + os.path.sep + 'Mask' + '_StopPerc_'+ str(StopPercentageSLADS), Mask)   
            np.save(trainingFeaturesSavePath + os.path.sep + 'ReconImage' + '_StopPerc_' + str(StopPercentageSLADS), ReconImage)
            np.save(trainingFeaturesSavePath + os.path.sep + 'PolyFeatures' + '_StopPerc_' + str(StopPercentageSLADS), PolyFeatures)

    print('\n\n\n\n\n' + ('-' * int(consoleColumns)))
    print('PERFORM TRAINING')
    print('-' * int(consoleColumns) + '\n')
    
    #Until the maximal SLADS stopping percentage
    #Append the observed polyfeatures for each sampling density and image to a single big array
    #NOTE: WHAT ABOUT LEAVING THESE IN MEMORY TO REDUCE TIME?
    for i in tqdm(range(0,len(c_vec)), desc = 'c Values', leave = True): #For each of the proposed c values
        c = c_vec[i]
        FirstLoop = 1
        for ImNum in tqdm(range(0,NumTrainingImages), desc = 'Training Images', leave = True): #For each of the possible training images

            for m in tqdm(range(0,np.size(MeasurementPercentageVector)), desc = 'Sampling Densities', leave = consoleRunning): #For each of the proposed sampling densities
                #Set loading paths
                LoadPath = trainingDataPath + 'FeaturesRegressCoeffs' + os.path.sep + 'Image_' + str(ImNum+1) + '_SampPerc_' + str(MeasurementPercentageVector[m])
                LoadPath_c = LoadPath + os.path.sep + 'c_' + str(c)

                #Load the image polynomial feature 
                PolyFeatures = np.load(LoadPath + os.path.sep + 'PolyFeatures' + '_StopPerc_' + str(StopPercentageSLADS)+ '.npy')

                #Load the possible reduction in distortion value
                RD = np.load(LoadPath_c + os.path.sep + 'RD' + '_StopPerc_' + str(StopPercentageSLADS)+ '.npy')

                if ImageType=='D':
                    if FirstLoop==1:
                        BigPolyFeatures = np.column_stack((PolyFeatures[:,0:25],PolyFeatures[:,26]))
                        BigRD = RD
                        FirstLoop = 0                  
                    else:
                        TempPolyFeatures = np.column_stack((PolyFeatures[:,0:25],PolyFeatures[:,26]))                    
                        BigPolyFeatures = np.row_stack((BigPolyFeatures,TempPolyFeatures))
                        BigRD = np.append(BigRD,RD)

                else: #If image type is continuous
                    if FirstLoop==1: #For the first iteration
                        BigPolyFeatures = PolyFeatures
                        BigRD = RD
                        FirstLoop = 0                  
                    else:
                        TempPolyFeatures = PolyFeatures               
                        BigPolyFeatures = np.row_stack((BigPolyFeatures,TempPolyFeatures))
                        BigRD = np.append(BigRD,RD)                    

        regr = linear_model.LinearRegression()

        #Perform the regression, fitting the observed polynomial fetures to the expected reduction in distortion
        regr.fit(BigPolyFeatures, BigRD)

        Theta = np.zeros((PolyFeatures.shape[1]))    

        if ImageType=='D':            
            Theta[0:24]=regr.coef_[0:24]
            Theta[26]=regr.coef_[25]
        else: #If image type is continuous
            Theta = regr.coef_

        del BigRD,BigPolyFeatures

        SavePath_c = trainingDataPath + 'c_' + str(c)
        if not os.path.exists(SavePath_c):
            os.makedirs(SavePath_c) 
        np.save(SavePath_c + os.path.sep + 'Theta' + '_StopPerc_' + str(StopPercentageSLADS), Theta)

    print('\n\n\n\n' + ('-' * int(consoleColumns)))
    print('DETERMINING BEST C')
    print('-' * int(consoleColumns) + '\n')

    #Until the maximal SLADS stopping percentage
    UpdateERDParamsObject = UpdateERDParams()
    UpdateERDParamsObject.initialize(Update_ERD,MinWindSize,MaxWindSize,1.5)

    # Find the best value of c
    if not overrideBestC:
        Best_c,NumImagesForSLADS = performSLADStoFindC(num_threads, consoleRunning, codePath,trainingDataPath, 
                                                   TrainingImageSet,ImageType,ImageExtension,TrainingInfoObject,
                                                   SizeImage,StopPercentageSLADS,Resolution,c_vec,
                                                   UpdateERDParamsObject,InitialMaskObject,MaskType,
                                                   reconPercVector,Classify,consoleRows,cPlot,
                                                   trainingStatisticsSavePath,lineScanning,
                                                   BatchSamplingParamsObject)

    #Set path to where best c value should be kept
    SavePath_bestc = trainingDataPath + 'best_c' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy'
    np.save(SavePath_bestc, np.array([Best_c]))

    #Directory checking
    ThetaSavePath = trainingFeaturesPath + TrainingDBName + os.path.sep + 'c_' + str(Best_c) + os.path.sep
    ThetaLoadPath = trainingFeaturesPath + TrainingDBName + os.path.sep + 'c_' + str(Best_c) + os.path.sep
    if not os.path.exists(ThetaSavePath):
        os.makedirs(ThetaSavePath)

    np.save(ThetaSavePath + 'Theta' + '_StopPerc_'+ str(StopPercentageSLADS), Theta)

    # Find the Threshold on stopping condition that corresponds to the desired total distortion (TD) value set above
    if FindStopThresh=='Y':   
        Threshold=findStoppingThreshold(trainingDataPath,NumImagesForSLADS,Best_c,PercentageInitialMask,DesiredTD,reconPercVector,SizeImage)
        #print('For a TD of '+ str(DesiredTD) + ' set stopping function threshold to: ' + str(Threshold))
        #print('**** Make sure to enter this value in runSimulations.py and in runSLADS.py')
        #print('The threshold value is saved in:  ' + ThetaSavePath + ' as Threshold.npy')
        np.save(ThetaSavePath + 'Threshold' + '_StopPerc_'+ str(StopPercentageSLADS), Threshold) 

    #Now that the optimal C value has been found for reconstruction
    #Train the model 1 image at a time, perform a reconstruction and watch statistics over all training images
    if (trainingErrorPlot):
        print('\n\n\n\n' + ('-' * int(consoleColumns)))
        print('PLOTTING MODEL TRAINING CONVERGENCE')
        print(('-' * int(consoleColumns)) + '\n')   
        plotXLabel = '# Training Samples' #x label for model training convergence plot
        errorPlot = True
        SavePath = './tmp'
        #Until the maximal SLADS stopping percentage
        Beta = computeBeta(SizeImage)
        StopCondParamsObject = StopCondParams()
        StopCondParamsObject.initialize(Beta,0,50,2,StopPercentageSLADS)

        PlotResult = 'N'
        loadPathInitialMask = resultsDataPath + 'InitialSamplingMasks' # Load initial measurement mask

        NumImagesPlot = len(testingImageFiles)
        trainingPlotTitle = 'Averaged Total Testing Image Results'

        #Create an object to hold progressive development of model
        meanSSIM = []
        meanMSE = []
        meanTD = []

        PlotResult = 'N'
        Idx_c = 0
        isRunningParallel = True

        loadPathImage = testingDataPath + 'Images' + os.path.sep
        loadPathInitialMask = resultsDataPath + 'InitialSamplingMasks'
        imageNames = glob.glob(loadPathImage + '*' + ImageExtension)
        NumImages = np.size(imageNames)

        #Import the best c and theta values; Training for database must have been performed first
        LoadPath_bestc = trainingFeaturesPath + 'TrainingDB_' + str(TestingImageSet) + os.path.sep + 'best_c' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy'
        if not os.path.exists(LoadPath_bestc):
            sys.exit('Error!!! The best c file ' + SavePath_bestc + ' does not exist. Check entry for ' + SavePath_bestc)
        c = np.load(LoadPath_bestc)[0].astype(float)
        TrainingInfoObject.FilterC=c

        FirstLoop = 1
        for ImNum in tqdm(range(0,NumTrainingImages), desc = 'Training Images', leave = True): #For each of the possible training images

            for m in tqdm(range(0,np.size(MeasurementPercentageVector)), desc = 'Sampling Densities', leave = consoleRunning): #For each of the proposed sampling densities
                #Set loading paths
                LoadPath = trainingDataPath + 'FeaturesRegressCoeffs' + os.path.sep + 'Image_' + str(ImNum+1) + '_SampPerc_' + str(MeasurementPercentageVector[m])
                LoadPath_c = LoadPath + os.path.sep + 'c_' + str(c)

                #Load the image polynomial feature 
                PolyFeatures = np.load(LoadPath + os.path.sep + 'PolyFeatures' + '_StopPerc_' + str(StopPercentageSLADS)+ '.npy')

                #Load the possible reduction in distortion value
                RD = np.load(LoadPath_c + os.path.sep + 'RD' + '_StopPerc_' + str(StopPercentageSLADS)+ '.npy')

                if ImageType=='D':
                    if FirstLoop==1:
                        BigPolyFeatures = np.column_stack((PolyFeatures[:,0:25],PolyFeatures[:,26]))
                        BigRD = RD
                        FirstLoop = 0                  
                    else:
                        TempPolyFeatures = np.column_stack((PolyFeatures[:,0:25],PolyFeatures[:,26]))                    
                        BigPolyFeatures = np.row_stack((BigPolyFeatures,TempPolyFeatures))
                        BigRD = np.append(BigRD,RD)

                else: #If image type is continuous
                    if FirstLoop==1: #For the first iteration
                        BigPolyFeatures = PolyFeatures
                        BigRD = RD
                        FirstLoop = 0                  
                    else:
                        TempPolyFeatures = PolyFeatures               
                        BigPolyFeatures = np.row_stack((BigPolyFeatures,TempPolyFeatures))
                        BigRD = np.append(BigRD,RD)                    

            regr = linear_model.LinearRegression()

            #Perform the regression, fitting the observed polynomial fetures to the expected reduction in distortion
            regr.fit(BigPolyFeatures, BigRD)

            Theta = np.zeros((PolyFeatures.shape[1]))    

            if ImageType=='D':            
                Theta[0:24]=regr.coef_[0:24]
                Theta[26]=regr.coef_[25]
            else: #If image type is continuous
                Theta = regr.coef_

            #Test the model for best c and the trained theta
            ssimResults = []
            mseResults = []
            tdResults = []

            results = Parallel(n_jobs=num_threads)(delayed(stats_parhelper)(InitialMaskObject, loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, SavePath, '', '', i, ImageExtension, Classify, isRunningParallel, lineScanning, testingData[i], False, False, True, True) for i in tqdm(range(0,len(testingData)), desc = 'Testing Images', leave = False))

            for result in results:
                ssimResults.append(result.ssimValues[len(result.ssimValues)-1])
                mseResults.append(result.mseValues[len(result.mseValues)-1])
                tdResults.append(result.tdValues[len(result.tdValues)-1])

            #Average the results
            meanSSIM.append(np.mean(ssimResults))
            meanMSE.append(np.mean(mseResults))
            meanTD.append(np.mean(tdResults))
        np.savetxt(trainingStatisticsSavePath+'meanSSIM.csv', meanSSIM, delimiter=',')
        np.savetxt(trainingStatisticsSavePath+'meanMSE.csv', meanMSE, delimiter=',')
        np.savetxt(trainingStatisticsSavePath+'meanTD.csv', meanTD, delimiter=',')


 
########################################
TRAINING MODEL
########################################









----------------------------------------
PERFORM TRAINING
----------------------------------------








----------------------------------------
DETERMINING BEST C
----------------------------------------








----------------------------------------
PLOTTING MODEL TRAINING CONVERGENCE
----------------------------------------



In [12]:
#RUN TESTING
#==================================================================  
if testingModel:
    print('\n\n' + ('#' * int(consoleColumns)))
    print('TESTING MODEL')
    print(('#' * int(consoleColumns)) + '\n')

    #Should the testing results be plotted
    PlotResult='Y'

    # If you want to use stopping function used, enter threshold (from Training), else leave at 0      
    StoppingThrehsold = 0

    #Static variables for SLADS function call
    
    #Directory setup for training database
    TrainingDBPath = trainingFeaturesPath + TrainingDBName
    if not os.path.exists(TrainingDBPath): 
        sys.exit('Error!!! The folder ' + TrainingDBPath + ' does not exist. Check entry for ' + TrainingImageSet)

    UpdateERDParamsObject = UpdateERDParams()
    UpdateERDParamsObject.initialize(Update_ERD,MinWindSize,MaxWindSize,1.5)
    PercentageInitialMask = float(PercentageInitialMask)
    np.save(testingFeaturesPath + 'InitialMask', Mask)
    savemat(testingFeaturesPath + 'InitialMask.mat', dict(Mask=Mask))
    testingPlotTitle = 'Averaged Total Testing Image Results'
    plotXLabel = '% Sampled'

    #Import the best c and theta values; Training for database must have been performed first
    if not overrideBestC: #If automatic best c selection
        LoadPath_bestc = trainingFeaturesPath + 'TrainingDB_' + str(TestingImageSet) + os.path.sep + 'best_c' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy'
        if not os.path.exists(LoadPath_bestc):
            sys.exit('Error!!! The best c file ' + SavePath_bestc + ' does not exist. Check entry for ' + SavePath_bestc)
        c = np.load(LoadPath_bestc)[0].astype(float)
    else: #If manual best c selection
        c = float(overrideBestCValue)

    #Directory setup for Theta
    ThetaLoadPath = trainingFeaturesPath + TrainingDBName + os.path.sep + 'c_' + str(c) + os.path.sep
    if not os.path.exists(ThetaLoadPath):                                                                                                                          
        sys.exit('Error!!! Check folder ./ResultsAndData/TrainingSavedFeatures/TrainingDB_' + TrainingImageSet + ' for folder c_' + str(c))

    #Load Theta, Beta, and stopping condtions
    Theta=np.transpose(np.load(ThetaLoadPath +'Theta' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy'))
    Beta = computeBeta(SizeImage)
    StopCondParamsObject=StopCondParams()
    StopCondParamsObject.initialize(Beta,StoppingThrehsold,50,0,StopPercentageTestingSLADS)
    
    #Obtain information on the testing set
    dataFileNames = glob.glob(testingDataImagesPath + "/*" + ImageExtension)
    numberTestFiles = len(testingData)

    animationFlag = True
    
    #Set path for saving the resulting images from the testing results
    testingImagesSavePath = testingSavePath + 'ImageResults' + os.path.sep    
    if os.path.exists(testingImagesSavePath):
        shutil.rmtree(testingImagesSavePath)  
    os.makedirs(testingImagesSavePath)

    AnimationImagesSavePath = testingImagesSavePath + 'Animation' + os.path.sep
    if os.path.exists(AnimationImagesSavePath): 
        shutil.rmtree(AnimationImagesSavePath)
    os.makedirs(AnimationImagesSavePath)

    #Perform SLADS on all of the testing images
    #testingResultObject = []
    #for i in tqdm(range(0,numberTestFiles), desc = 'Testing Images', leave = True):
    #    result = runSLADS(InitialMaskObject, loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, testingFeaturesPath, testingSavePath, re.sub('\.png', '', os.path.basename(dataFileNames[i])), i, ImageExtension, Classify, False, lineScanning, testingData[i], True, animationFlag, True)
    #    testingResultObject.append(result)
        
    #Perform SLADS on all of the images, saving statistics of interest in parallel
    testingResultObject = Parallel(n_jobs=num_threads)(delayed(stats_parhelper)(InitialMaskObject, loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, testingFeaturesPath, testingSavePath, re.sub('\.png', '', os.path.basename(dataFileNames[i])), i, ImageExtension, Classify, True, lineScanning, testingData[i], True, animationFlag, True,False) for i in tqdm(range(0,numberTestFiles), desc = 'Testing Images', leave = True))
    #testingResultObject = Parallel(n_jobs=num_threads)(delayed(stats_parhelper)(InitialMaskObject, Mask.copy(), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, testingFeaturesPath, testingSavePath, re.sub('\.png', '', os.path.basename(dataFileNames[i])), i, ImageExtension, Classify, True, lineScanning, testingData[i], True, animationFlag, True, False) for i in tqdm(range(0,numberTestFiles), desc = 'Testing Images', leave = True))
    
    #Create holding arrays for all of the results
    MSE_testingResults = []
    SSIM_testingResults = []
    TD_testingResults = []
    MSE_testingResults_forImage = []
    SSIM_testingResults_forImage = []
    TD_testingResults_forImage = []
    times_testingResults = []
    perc_testingResults = []
    
    #Extract results from returned object; for every line scanned there is a value
    for result in testingResultObject: 
        MSE_testingResults.append(result.mseValues)
        SSIM_testingResults.append(result.ssimValues)
        TD_testingResults.append(result.tdValues)
        MSE_testingResults_forImage.append(result.mseValues_forImage)
        SSIM_testingResults_forImage.append(result.ssimValues_forImage)
        TD_testingResults_forImage.append(result.tdValues_forImage)
        times_testingResults.append(result.timesToDetermine)
        perc_testingResults.append(result.percentageValues)

precision = 0.01

percents, testingSSIM_mean = perc_Results(SSIM_testingResults, perc_testingResults, precision)
np.savetxt(testingStatisticsSavePath+'testingPlotAverageSSIM_Percentage.csv', np.transpose([percents, testingSSIM_mean]), delimiter=',')
font = {'size' : 18}
plt.rc('font', **font)
f = plt.figure(figsize=(20,8))
ax1 = f.add_subplot(1,1,1)    
ax1.plot(percents,testingSSIM_mean,color='black') 
ax1.set_xlabel('% Pixels Measured')
ax1.set_ylabel('Average SSIM')
plt.savefig(testingStatisticsSavePath + 'testingPlotAverageSSIM_Percentage' + '.png')
plt.close()

#Find the final SSIM Results for each image
lastSSIMResult = []
percLinesScanned = []
percPixelsScanned = []
for i in range(0, len(SSIM_testingResults)):
    lastSSIMResult.append(SSIM_testingResults[i][len(SSIM_testingResults[i])-1])
    percLinesScanned.append((len(SSIM_testingResults[i])/len(linesToScan))*100)
    percPixelsScanned.append(perc_testingResults[i][len(perc_testingResults[i])-1])

ssimPrintout = []
ssimPrintout.append(['Average SSIM:',np.mean(lastSSIMResult), '+/-', np.std(lastSSIMResult)])
ssimPrintout.append(['Average % Lines Scanned:',np.mean(percLinesScanned),'+/-', np.std(percLinesScanned)])
ssimPrintout.append(['Average % Pixels Scanned:',np.mean(percPixelsScanned),'+/-',np.std(percPixelsScanned)])
pd.DataFrame(ssimPrintout).to_csv(testingStatisticsSavePath+'ssimPrintout.csv')


#AFTER INTENDED PROCEDURES (TRAINING/TESTING) HAVE BEEN PERFORMED
print('\n\n\n' + ('#' * int(consoleColumns)))
print('PROGRAM COMPLETE')
print('#' * int(consoleColumns) + '\n')






########################################
TESTING MODEL
########################################







########################################
PROGRAM COMPLETE
########################################



In [13]:
#Find the final Results for each image
lastSSIMResult = []
lastMSEResult = []
lastTDResult = []
percPixelsScanned = []
for i in range(0, len(SSIM_testingResults)):
    lastSSIMResult.append(SSIM_testingResults[i][len(SSIM_testingResults[i])-1])
    lastMSEResult.append(MSE_testingResults[i][len(MSE_testingResults[i])-1])
    lastTDResult.append(TD_testingResults[i][len(TD_testingResults[i])-1])
    percPixelsScanned.append(perc_testingResults[i][len(perc_testingResults[i])-1])


printout = []
printout.append(['Average SSIM:',np.mean(lastSSIMResult), '+/-', np.std(lastSSIMResult)])
printout.append(['Average MSE:',np.mean(lastMSEResult), '+/-', np.std(lastMSEResult)])
printout.append(['Average TD:',np.mean(lastTDResult), '+/-', np.std(lastTDResult)])
printout.append(['Average %:',np.mean(percPixelsScanned), '+/-', np.std(percPixelsScanned)])
pd.DataFrame(printout).to_csv(testingStatisticsSavePath+'printout.csv')


In [ ]:

    #Determine the average result values across all testing images
    TDAverageResults, TDSTDResults = resultsAverager(TD_testingResults)
    MSEAverageResults, MSESTDResults = resultsAverager(MSE_testingResults)
    SSIMAverageResults, SSIMSTDResults = resultsAverager(SSIM_testingResults)
    TDAverageResults_forImage, TDSTDResults_forImage = resultsAverager(TD_testingResults_forImage)
    MSEAverageResults_forImage, MSESTDResults_forImage = resultsAverager(MSE_testingResults_forImage)
    SSIMAverageResults_forImage, SSIMSTDResult_forImage = resultsAverager(SSIM_testingResults_forImage)
    timesAverageResults, timesSTDResults = resultsAverager(times_testingResults)
    percAverageResults, percSTDResults = resultsAverager(perc_testingResults)
    
    #Determine the number of rows for each of the results
    #numRows = []
    #for i in range(0,len(MSE_testingResults)):
    #    numRows.append(len(MSE_testingResults[i]))
    
    #Create placeholders for averages across all results
    #TDAverageResults = []
    #MSEAverageResults = []
    #SSIMAverageResults = []
    #TDAverageResults_forImage = []
    #MSEAverageResults_forImage = []
    #SSIMAverageResults_forImage = []
    #timesAverageResults = []
    #percAverageResults = []
    #for i in range(0, max(numRows)-1):
    #    TDSumResults = 0
    #    MSESumResults = 0
    #    SSIMSumResults = 0
    #    TDSumResults_forImage = 0
    #    MSESumResults_forImage = 0
    #    SSIMSumResults_forImage = 0
    #    timesSumResults = 0
    #    percSumResults = 0
    #    lenSum = 0
    #    for j in range(0,len(numRows)):
    #        if i < numRows[j]-1: #If the testing image had still been scanning rows
    #            TDSumResults += TD_testingResults[j][i]
    #            MSESumResults += MSE_testingResults[j][i]
    #            SSIMSumResults += SSIM_testingResults[j][i]
    #            TDSumResults_forImage += TD_testingResults_forImage[j][i]
    #            MSESumResults_forImage += MSE_testingResults_forImage[j][i]
    #            SSIMSumResults_forImage += SSIM_testingResults_forImage[j][i]
    #            timesSumResults += times_testingResults[j][i]
    #            percSumResults += perc_testingResults[j][i]
    #            lenSum += 1
    #    TDAverageResults.append(TDSumResults/lenSum)
    #    MSEAverageResults.append(MSESumResults/lenSum)
    #    SSIMAverageResults.append(SSIMSumResults/lenSum)
    #    TDAverageResults_forImage.append(TDSumResults_forImage/lenSum)
    #    MSEAverageResults_forImage.append(MSESumResults_forImage/lenSum)
    #    SSIMAverageResults_forImage.append(SSIMSumResults_forImage/lenSum)
    #    timesAverageResults.append(timesSumResults/lenSum)
    #    percAverageResults.append(percSumResults/lenSum)        

    print('\n\n' + ('#' * int(consoleColumns)))
    print('PLOTTING STATISTICS')
    print(('#' * int(consoleColumns)) + '\n')
    
    #SSIM - AGAINST PERCENTAGE PIXELS MEASURED
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)    
    ax1.plot(percAverageResults,SSIMAverageResults,'.',color='black',label='Ground-Truth') 
    ax1.plot(percAverageResults,SSIMAverageResults_forImage,'*',color='black',label='Foreground')     
    plt.legend()
    ax1.set_xlabel('Mean % Pixels Measured')
    ax1.set_ylabel('Mean SSIM')
    plt.savefig(testingStatisticsSavePath + 'testingPlotAverageSSIM_Percentage' + '.png')
    plt.close()
    
    #MSE - AGAINST PERCENTAGE PIXELS MEASURED
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)
    ax1.plot(percAverageResults,MSEAverageResults,'.',color='black',label='Ground-Truth') 
    ax1.plot(percAverageResults,MSEAverageResults_forImage,'*',color='black',label='Foreground') 
    ax1.legend()
    ax1.set_xlabel('Mean % Pixels Measured')
    ax1.set_ylabel('Mean MSE')
    plt.savefig(testingStatisticsSavePath + 'testingPlotAverageMSE_Percentage' + '.png')
    plt.close()

    #TD - AGAINST PERCENTAGE PIXELS MEASURED
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)
    ax1.plot(percAverageResults,TDAverageResults,'.',color='black',label='Ground-Truth') 
    ax1.plot(percAverageResults,TDAverageResults_forImage,'*',color='black',label='Foreground') 
    ax1.legend()
    ax1.set_xlabel('Mean % Pixels Measured')
    ax1.set_ylabel('Mean Total Distortion')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_AverageTotalDistortion_Percentage' + '.png')
    plt.close()

    #Average search time  - AGAINST PERCENTAGE PIXELS MEASURED
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)
    ax1.plot(percAverageResults, timesAverageResults,'.',color='black') 
    ax1.set_xlabel('Mean % Pixels Measured')
    ax1.set_ylabel('Mean Search Time (s)')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_AverageSeachTime_Percentage' + '.png')
    plt.close()

    #Percent scanned per line
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()
    plt.plot(percAverageResults, color='black') 
    plt.plot(percAverageResults, '.', color='black') 
    plt.xlabel('# of New Measurement Selections')
    plt.ylabel('% Pixels Scanned')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_PercPerLine' + '.png')
    plt.close()
    
    #Percent scanned per line
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot()
    plt.plot(timesAverageResults, color='black') 
    plt.plot(timesAverageResults, '.', color='black') 
    plt.xlabel('# of New Measurement Selections')
    plt.ylabel('Selection Time (s)')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_TimeToSelection' + '.png')
    plt.close()

    #SSIM - AGAINST OF NEW MEASUREMENT SELECTIONS
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)    
    ax1.plot(SSIMAverageResults, color='black',label='Ground-Truth') 
    ax1.plot(SSIMAverageResults,'.',color='black') 
    ax1.plot(SSIMAverageResults_forImage, '--',color='black',label='Foreground') 
    ax1.plot(SSIMAverageResults_forImage,'.',color='black')     
    plt.legend()
    ax1.set_xlabel('# of New Measurement Selections')
    ax1.set_ylabel('Mean SSIM')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_AverageSSIM_numMeasurements' + '.png')
    plt.close()
    
    #MSE - AGAINST OF NEW MEASUREMENT SELECTIONS
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)
    ax1.plot(MSEAverageResults, color='black',label='Ground-Truth') 
    ax1.plot(MSEAverageResults,'.',color='black') 
    ax1.plot(MSEAverageResults_forImage, '--',color='black',label='Foreground') 
    ax1.plot(MSEAverageResults_forImage,'.',color='black') 
    ax1.legend()
    ax1.set_xlabel('# of New Measurement Selections')
    ax1.set_ylabel('Mean MSE')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_AverageMSE_numMeasurements' + '.png')
    plt.close()

    #TD - AGAINST OF NEW MEASUREMENT SELECTIONS
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)
    ax1.plot(TDAverageResults, color='black',label='Ground-Truth') 
    ax1.plot(TDAverageResults,'.',color='black') 
    ax1.plot(TDAverageResults_forImage, '--',color='black',label='Foreground') 
    ax1.plot(TDAverageResults_forImage,'.',color='black') 
    ax1.legend()
    ax1.set_xlabel('# of New Measurement Selections')
    ax1.set_ylabel('Mean Total Distortion')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_AverageTotalDistortion_numMeasurements' + '.png')
    plt.close()

    #Average search time - AGAINST OF NEW MEASUREMENT SELECTIONS
    font = {'size' : 18}
    plt.rc('font', **font)
    f = plt.figure(figsize=(20,8))
    ax1 = f.add_subplot(1,1,1)
    ax1.plot(timesAverageResults, color='black',label='Ground-Truth') 
    ax1.plot(timesAverageResults,'.',color='black') 
    ax1.set_xlabel('# of New Measurement Selections')
    ax1.set_ylabel('Mean Search Time (s)')
    plt.savefig(testingStatisticsSavePath + 'testingPlot_AverageSeachTime_numMeasurements' + '.png')
    plt.close()
